In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
from sklearn.cluster import KMeans
from pandas_datareader import data
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import plotly
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.formula.api as smf
#import getFamaFrenchFactors as gff

### RISK-FREE RATES

In [2]:
risk_free=pd.read_csv("/Users/USER/Documents/DATASETS/TreasuryBill.csv")[['TreasuryBill','Date']]

In [3]:
risk_free['Date']=pd.to_datetime(risk_free['Date'])

In [4]:
risk_free.columns=risk_free.columns.str.replace('Date','datetime')

In [5]:
risk_free=risk_free.set_index('datetime')

In [6]:
risk_free=risk_free.resample('1M').last()

In [7]:
risk_free=risk_free.pct_change().dropna(how='all')

In [8]:
risk_free=risk_free.reset_index()

In [9]:
risk_free

,datetime,TreasuryBill
0,2006-02-28,-0.160088
1,2006-03-31,-0.141862
2,2006-04-30,-0.127789
3,2006-05-31,-0.226744
4,2006-06-30,0.264662
...,...,...
193,2022-03-31,-0.297189
194,2022-04-30,-0.005714
195,2022-05-31,0.419540
196,2022-06-30,-0.024291


### MONTHLY STOCK PRICE DATA

In [10]:
data=pd.read_csv("/Users/USER/Documents/DATASETS/NSE_MONTHLY_Stock_data_2012_2022.csv")

In [11]:
data=data[108:].reset_index(drop=True)#2021 to 2022

In [12]:
new_df=data.set_index('datetime')

In [13]:
new_df=new_df.pct_change().dropna(how='all')

In [14]:
new_df

,AIICO,FTNCOCOA,LIVINGTRUST,ABCTRANS,ACADEMY,AFRINSURE,AFROMEDIA,AIRTELAFRI,ALEX,BERGER,...,MBENEFIT,MEYER,MORISON,MRS,NEM,UACN,UCAP,UNIVINSURE,WAPIC,VITAFOAM
datetime,,,,,,,,,,,,,,,,,,,,,
2021-02-01 10:15:00,-0.178570,-0.109091,0.133333,-0.028571,0.242424,-0.040000,0.0,0.000000,0.012500,-0.099379,...,-0.069767,0.000000,0.100000,0.089431,-0.173554,-0.101796,0.038983,0.000000,0.000000,-0.228856
2021-03-01 10:15:00,0.086956,-0.061224,0.000000,-0.117647,0.000000,-0.166667,0.0,0.000000,0.000000,-0.103448,...,0.150000,-0.100000,0.563636,-0.097015,0.150000,0.200000,-0.135400,0.000000,-0.074074,0.070968
2021-04-01 10:15:00,0.032000,-0.086957,0.000000,0.033333,-0.024390,0.000000,0.0,0.000000,-0.098765,0.000000,...,-0.152174,0.155556,0.093023,-0.099174,-0.117391,0.144444,0.052830,0.000000,0.060000,0.084337
2021-05-04 10:15:00,-0.069768,-0.095238,-0.014706,0.096774,-0.175000,0.000000,0.0,-0.100000,0.000000,-0.061538,...,0.128205,0.096154,0.000000,0.155963,0.019704,0.067961,0.107527,0.000000,0.000000,0.222222
2021-06-01 10:15:00,-0.133333,-0.131579,0.000000,0.088235,-0.030303,0.000000,0.0,-0.202270,-0.013699,0.377049,...,-0.022727,0.035088,0.489362,0.000000,-0.072464,-0.113636,0.003236,0.000000,0.075472,0.400000
2021-07-01 10:15:00,0.038462,0.454545,0.000000,-0.027027,0.187500,0.000000,0.0,-0.078928,0.000000,0.059524,...,-0.139535,0.000000,0.000000,0.000000,0.041667,0.046154,0.129032,0.000000,-0.035088,-0.025974
2021-08-02 10:15:00,-0.083335,0.187500,0.000000,-0.055556,0.026316,0.000000,0.0,0.219512,0.000000,0.000000,...,-0.135135,-0.661017,0.450000,0.099206,-0.030000,0.078431,0.025714,0.000000,-0.109091,0.100000
2021-09-01 10:15:00,-0.040404,-0.140351,0.000000,-0.029412,0.000000,0.000000,0.0,-0.046667,0.000000,0.000000,...,-0.062500,0.000000,0.034483,0.097473,-0.015464,-0.068182,0.246518,0.000000,-0.020408,0.036364
2021-10-04 10:15:00,0.305265,-0.102041,0.194030,-0.060606,-0.153846,0.000000,0.0,0.090909,0.000000,0.050562,...,-0.066667,0.000000,-0.100000,0.000000,0.047120,0.102439,0.027933,0.100000,0.000000,0.081871


### CALCULATING HML AND SMB

**BOOK-TO-MARKET AND MARKET-CAP**

     Market Capitalization= Price * Total shares outstanding
     
     Price-to-book = Market cap/ (total assets - total liabilities)
     
     book-to-market-ratio=1/Price-to-book
     
     book-to-market-ratio = (total assets - total liabilities) / Market cap

In [15]:
df=pd.read_excel("/Users/USER/Documents/DATASETS/FAMA_FRENCH/MANSARD.xlsx")

In [16]:
df

,Date,mansard_prices,mansard_OS,mansard_TA,mansard_TL,DATA SOURCE
0,9/30/2021,2.33,9000000000,96007000000,60070000000,WSJ.COM
1,10/31/2021,2.35,9000000000,96007000000,60070000000,WSJ.COM
2,11/30/2021,2.12,9000000000,96007000000,60070000000,WSJ.COM
3,12/31/2021,2.32,9000000000,92892000000,58499000000,WSJ.COM
4,1/31/2022,2.35,9000000000,92892000000,58499000000,WSJ.COM
5,2/28/2022,2.40,9000000000,92892000000,58499000000,WSJ.COM
6,3/31/2022,2.37,8944260000,102496000000,66642000000,WSJ.COM
7,4/30/2022,2.44,8944260000,102496000000,66642000000,WSJ.COM
8,5/31/2022,2.14,8944260000,102496000000,66642000000,WSJ.COM
9,6/30/2022,2.00,8944260000,104473000000,68858000000,WSJ.COM


In [17]:
def calculate_values(filepath):
    """A function that imports the datasets and calculates it's book-to-market ratio and market capitalization"""
    df=pd.read_excel(filepath)
    df['market_cap']=(df.iloc[:,1]) * (df.iloc[:,2])
    df['book_to_market_ratio']=((df.iloc[:,3]) - (df.iloc[:,4])) / df['market_cap']
    return df

In [18]:
MANSARD=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/MANSARD.xlsx")
AIICO=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/AIICO.xlsx")
FTNCOCOA=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/FTNCOCOA.xlsx")
LIVINGTRUST=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/LIVINGTRUST.xlsx")
ABCTRANS=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/ABCTRANS.xlsx")
ACADEMY=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/ACADEMY.xlsx")
AFRINSURE=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/AFRINSURE.xlsx")
AFROMEDIA=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/AFROMEDIA.xlsx")
AIRTELAFRI=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/AIRTELAFRI.xlsx")
ALEX=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/ALEX.xlsx")
BERGER=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/BERGER.xlsx")
BETAGLAS=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/BETAGLAS.xlsx")
BUACEMENT=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/BUACEMENT.xlsx")
BUAFOODS=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/BUAFOODS.xlsx")
CADBURY=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/CADBURY.xlsx")
CAP=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/CAP.xlsx")
CAVERTON=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/CAVERTON.xlsx")
CHAMPION=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/CHAMPION.xlsx")
CHAMS=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/CHAMS.xlsx")
CHELLARAM=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/CHELLARAM.xlsx")
CHIPLC=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/CHIPLC.xlsx")
CILEASING=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/CILEASING.xlsx")
ETRANZACT=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/ETRANZACT.xlsx")
CONOIL=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/CONOIL.xlsx")
TRANSCORP=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/TRANSCORP.xlsx")
UNILEVER=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/UNILEVER.xlsx")
UPDCREIT=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/UPDCREIT.xlsx")
UPL=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/UPL.xlsx")
VERITASKAP=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/VERITASKAP.xlsx")
ZENITHBANK=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/ZENITHBANK.xlsx")
WAPCO=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/WAPCO.xlsx")
UPDC=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/UPDC.xlsx")
UBN=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/UBN.xlsx")
PZ=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/PZ.xlsx")
LIVESTOCK=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/LIVESTOCK.xlsx")
JAIZBANK=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/JAIZBANK.xlsx")
FCMB=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/FCMB.xlsx")
FIDELITYBK=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/FIDELITYBK.xlsx")
UBA=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/UBA.xlsx")
TOTAL=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/TOTAL.xlsx")
STANBIC=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/STANBIC.xlsx")
SOVRENINS=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/SOVRENINS.xlsx")
RTBRISCOE=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/RTBRISCOE.xlsx")
SCOA=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/SCOA.xlsx")

In [19]:
REDSTAREX=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/REDSTAREX.xlsx")
REGALINS=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/REGALINS.xlsx")
PRESCO=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/PRESCO.xlsx")
PHARMDEKO=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/PHARMDEKO.xlsx")
OANDO=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/OANDO.xlsx")
OKOMUOIL=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/OKOMUOIL.xlsx")
NNFM=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/NNFM.xlsx")
NGXGROUP=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/NGXGROUP.xlsx")
NESTLE=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/NESTLE.xlsx")
NEIMETH=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/NEIMETH.xlsx")
NB=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/NB.xlsx")
MULTIVERSE=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/MULTIVERSE.xlsx")
NAHCO=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/NAHCO.xlsx")
NASCON=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/NASCON.xlsx")
MTNN=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/MTNN.xlsx")
MAYBAKER=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/MAYBAKER.xlsx")
MCNICHOLS=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/MCNICHOLS.xlsx")
LEARNAFRCA=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/LEARNAFRCA.xlsx")
LINKASSURE=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/LINKASSURE.xlsx")
INTBREW=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/INTBREW.xlsx")
GUINNESS=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/GUINESS.xlsx")
HONYFLOUR=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/HONYFLOUR.xlsx")
GLAXOSMITH=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/GLAXOSMITH.xlsx")
FBNH=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/FBNH.xlsx")
AFRIPRUD=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/AFRIPRUD.xlsx")
COURTVILLE=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/COURTVILLE.xlsx")
CUSTODIAN=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/CUSTODIAN.xlsx")
CWG=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/CWG.xlsx")
DANGCEM=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/DANGCEM.xlsx")
ELLAHLAKES=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/ELLAHLAKES.xlsx")
ETERNA=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/ETERNA.xlsx")
ETI=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/ETI.xlsx")
FLOURMILL=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/FLOURMILL.xlsx")
LASACO=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/LASACO.xlsx")
MANSARD=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/MANSARD.xlsx")
MBENEFIT=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/MBENEFIT.xlsx")
MEYER=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/MEYER.xlsx")
MORISON=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/MORISON.xlsx")
MRS=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/MRS.xlsx")
NEM=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/NEM.xlsx")
UACN=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/UACN.xlsx")
UCAP=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/UCAP.xlsx")
UNIVINSURE=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/UNIVINSURE.xlsx")
WAPIC=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/WAPIC.xlsx")
VITAFOAM=calculate_values("/Users/USER/Documents/DATASETS/FAMA_FRENCH/VITAFOAM.xlsx")

In [20]:
VITAFOAM,SCOA

(          Date  vitafoam_prices  vitafoam_OS  vitafoam_TA  vitafoam_TL  \
 0    9/30/2021            17.10   1250840000  31789670000  19635060000   
 1   10/31/2021            18.50   1250840000  31789670000  19635060000   
 2   11/30/2021            21.90   1250840000  31789670000  19635060000   
 3   12/31/2021            22.50   1250840000  34374010000  20722240000   
 4    1/31/2022            24.40   1250840000  34374010000  20722240000   
 5    2/28/2022            22.30   1250840000  34374010000  20722240000   
 6    3/31/2022            22.30   1250840000  37474070000  23799430000   
 7    4/30/2022            24.25   1250840000  37474070000  23799430000   
 8    5/31/2022            22.55   1250840000  37474070000  23799430000   
 9    6/30/2022            22.50   1250840000  36116300000  20654500000   
 10   7/30/2022            20.25   1250840000  36116300000  20654500000   
 11   8/31/2022            23.60   1250840000  36116300000  20654500000   
 
       DATA SOURCE    m

In [21]:
REDSTAREX.iloc[0,7],AIICO.iloc[0,27]

(1.5772224321260861, 2.402541649933678)

In [22]:
UACN

,Date,uacn_prices,uacn_OS,uacn_TA,uacn_TL,DATA SOURCE,market_cap,book_to_market_ratio
0,9/30/2021,10.25,2881300000,8719400000,37265000000,INVESTING.COM,2.953332e+10,-0.966556
1,10/31/2021,11.30,2881300000,8719400000,37265000000,INVESTING.COM,3.255869e+10,-0.876743
2,11/30/2021,10.00,2881300000,8719400000,37265000000,INVESTING.COM,2.881300e+10,-0.990719
3,12/31/2021,9.50,2881300000,98463000000,47625000000,INVESTING.COM,2.737235e+10,1.857276
4,1/31/2022,8.90,2881300000,98463000000,47625000000,INVESTING.COM,2.564357e+10,1.982485
5,2/28/2022,10.25,2881300000,98463000000,47625000000,INVESTING.COM,2.953332e+10,1.721377
6,3/31/2022,11.95,2881300000,115449000000,63971000000,INVESTING.COM,3.443154e+10,1.495083
7,4/30/2022,12.00,2881300000,115449000000,63971000000,INVESTING.COM,3.457560e+10,1.488853
8,5/31/2022,11.50,2881300000,115449000000,63971000000,INVESTING.COM,3.313495e+10,1.553586
9,6/30/2022,11.10,2881300000,115449000000,63971000000,INVESTING.COM,3.198243e+10,1.609571


In [23]:
price=new_df.iloc[7,:].values.flatten().tolist()#prices from new_df,  monthly stock variable
september_2021=pd.DataFrame({'Market_Cap':[AIICO.iloc[0,26],FTNCOCOA.iloc[0,26],LIVINGTRUST.iloc[0,26],ABCTRANS.iloc[0,26],
                                           ACADEMY.iloc[0,26],AFRINSURE.iloc[0,26],AFROMEDIA.iloc[0,6],AIRTELAFRI.iloc[0,26],
                                           ALEX.iloc[0,26],BERGER.iloc[0,26],BETAGLAS.iloc[0,26],BUACEMENT.iloc[0,26],
                                           BUAFOODS.iloc[0,26],CADBURY.iloc[0,26],CAP.iloc[0,26],CAVERTON.iloc[0,26],
                                           CHAMPION.iloc[0,26],CHAMS.iloc[0,26],CHELLARAM.iloc[0,26],CHIPLC.iloc[0,26],
                                           CILEASING.iloc[0,26],ETRANZACT.iloc[0,26],CONOIL.iloc[0,26],TRANSCORP.iloc[0,26],
                                           UNILEVER.iloc[0,26],UPDCREIT.iloc[0,26],UPL.iloc[0,26],VERITASKAP.iloc[0,26],
                                           ZENITHBANK.iloc[0,26],WAPCO.iloc[0,26],UPDC.iloc[0,26],UBN.iloc[0,26],
                                           PZ.iloc[0,26],LIVESTOCK.iloc[0,26],JAIZBANK.iloc[0,26],FCMB.iloc[0,26],
                                           FIDELITYBK.iloc[0,26],UBA.iloc[0,26],TOTAL.iloc[0,26],STANBIC.iloc[0,26],
                                           SOVRENINS.iloc[0,26],RTBRISCOE.iloc[0,26],SCOA.iloc[0,26],REDSTAREX.iloc[0,6],
                                           REGALINS.iloc[0,6],PRESCO.iloc[0,6],PHARMDEKO.iloc[0,6],OANDO.iloc[0,6],
                                           OKOMUOIL.iloc[0,6],NNFM.iloc[0,6],NGXGROUP.iloc[0,6],NESTLE.iloc[0,6],
                                           NEIMETH.iloc[0,6],NB.iloc[0,6],MULTIVERSE.iloc[0,6],NAHCO.iloc[0,6],NASCON.iloc[0,6],
                                           MTNN.iloc[0,6],MAYBAKER.iloc[0,6],MCNICHOLS.iloc[0,6],LEARNAFRCA.iloc[0,6],
                                           LINKASSURE.iloc[0,6],INTBREW.iloc[0,6],GUINNESS.iloc[0,6],HONYFLOUR.iloc[0,6],
                                           GLAXOSMITH.iloc[0,6],FBNH.iloc[0,6],AFRIPRUD.iloc[0,6],COURTVILLE.iloc[0,6],
                                           CUSTODIAN.iloc[0,6],CWG.iloc[0,6],DANGCEM.iloc[0,6],ELLAHLAKES.iloc[0,6],
                                           ETERNA.iloc[0,6],ETI.iloc[0,6],FLOURMILL.iloc[0,6],LASACO.iloc[0,6],
                                           MANSARD.iloc[0,6],MBENEFIT.iloc[0,6],MEYER.iloc[0,6],MORISON.iloc[0,6],MRS.iloc[0,6],
                                           NEM.iloc[0,6],UACN.iloc[0,6],UCAP.iloc[0,6],UNIVINSURE.iloc[0,6],WAPIC.iloc[0,6],
                                           VITAFOAM.iloc[0,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[0,27],FTNCOCOA.iloc[0,27],LIVINGTRUST.iloc[0,27],
                                                     ABCTRANS.iloc[0,27],ACADEMY.iloc[0,27],AFRINSURE.iloc[0,27],
                                                     AFROMEDIA.iloc[0,7],AIRTELAFRI.iloc[0,27],ALEX.iloc[0,27],BERGER.iloc[0,27],
                                                     BETAGLAS.iloc[0,27],BUACEMENT.iloc[0,27],BUAFOODS.iloc[0,27],
                                                     CADBURY.iloc[0,27],CAP.iloc[0,27],CAVERTON.iloc[0,27],CHAMPION.iloc[0,27],
                                                     CHAMS.iloc[0,27],CHELLARAM.iloc[0,27],CHIPLC.iloc[0,27],CILEASING.iloc[0,27],
                                                     ETRANZACT.iloc[0,27],CONOIL.iloc[0,27],TRANSCORP.iloc[0,27],
                                                     UNILEVER.iloc[0,27],UPDCREIT.iloc[0,27],UPL.iloc[0,27],VERITASKAP.iloc[0,27],
                                                     ZENITHBANK.iloc[0,27],WAPCO.iloc[0,27],UPDC.iloc[0,27],UBN.iloc[0,27],
                                                     PZ.iloc[0,27],LIVESTOCK.iloc[0,27],JAIZBANK.iloc[0,27],FCMB.iloc[0,27],
                                                     FIDELITYBK.iloc[0,27],UBA.iloc[0,27],TOTAL.iloc[0,27],STANBIC.iloc[0,27],
                                                     SOVRENINS.iloc[0,27],RTBRISCOE.iloc[0,27],SCOA.iloc[0,27],
                                                     REDSTAREX.iloc[0,7],
                                                     REGALINS.iloc[0,7],PRESCO.iloc[0,7],PHARMDEKO.iloc[0,7],OANDO.iloc[0,7],
                                                     OKOMUOIL.iloc[0,7],NNFM.iloc[0,7],NGXGROUP.iloc[0,7],NESTLE.iloc[0,7],
                                                     NEIMETH.iloc[0,7],NB.iloc[0,7],MULTIVERSE.iloc[0,7],NAHCO.iloc[0,7],
                                                     NASCON.iloc[0,7],MTNN.iloc[0,7],MAYBAKER.iloc[0,7],MCNICHOLS.iloc[0,7],
                                                     LEARNAFRCA.iloc[0,7],LINKASSURE.iloc[0,7],INTBREW.iloc[0,7],
                                                     GUINNESS.iloc[0,7],HONYFLOUR.iloc[0,7],GLAXOSMITH.iloc[0,7],FBNH.iloc[0,7],
                                                     AFRIPRUD.iloc[0,7],COURTVILLE.iloc[0,7],CUSTODIAN.iloc[0,7],CWG.iloc[0,7],
                                                     DANGCEM.iloc[0,7],ELLAHLAKES.iloc[0,7],ETERNA.iloc[0,7],ETI.iloc[0,7],
                                                     FLOURMILL.iloc[0,7],LASACO.iloc[0,7],MANSARD.iloc[0,7],MBENEFIT.iloc[0,7],
                                                     MEYER.iloc[0,7],MORISON.iloc[0,7],MRS.iloc[0,7],NEM.iloc[0,7],
                                                     UACN.iloc[0,7],UCAP.iloc[0,7],UNIVINSURE.iloc[0,7],WAPIC.iloc[0,7],
                                                     VITAFOAM.iloc[0,7]],
                             
                            
                            'Change_in_Price':price},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [24]:
september_2021

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,1.490359e+10,2.402542,-0.040404
FTNCOCOA,1.078000e+09,-0.112328,-0.140351
LIVINGTRUST,3.350000e+09,0.814084,0.000000
ABCTRANS,5.470410e+08,2.014109,-0.029412
ACADEMY,2.358720e+08,1.695623,0.000000
...,...,...,...
UACN,2.953332e+10,-0.966556,-0.068182
UCAP,5.370000e+10,0.500727,0.246518
UNIVINSURE,3.200000e+09,2.903950,0.000000
WAPIC,1.151606e+10,2.000163,-0.020408


In [25]:
SL=september_2021[(september_2021['Market_Cap']<=2000000000)&(september_2021['Book_to_market_ratio']>1)]

SM=september_2021[(september_2021['Market_Cap']>2000000000) & (september_2021['Market_Cap']<10000000000) & (september_2021['Book_to_market_ratio']>1)]

SH=september_2021[(september_2021['Market_Cap']>=10000000000)&(september_2021['Book_to_market_ratio']>1)]

BL=september_2021[(september_2021['Market_Cap']<=2000000000) & (september_2021['Book_to_market_ratio']<1)]

BH=september_2021[(september_2021['Market_Cap']>=10000000000)&(september_2021['Book_to_market_ratio']<1)]

BM=september_2021[(september_2021['Market_Cap']>2000000000) & (september_2021['Market_Cap']<10000000000) & (september_2021['Book_to_market_ratio']<1)]


In [26]:
SLPRICE=SL['Change_in_Price'].mean()

SMPRICE=SM['Change_in_Price'].mean()

SHPRICE=SH['Change_in_Price'].mean()

BLPRICE=BL['Change_in_Price'].mean()

BMPRICE=BM['Change_in_Price'].mean()

BHPRICE=BH['Change_in_Price'].mean()



In [27]:
SM

,Market_Cap,Book_to_market_ratio,Change_in_Price
BERGER,2.579398e+09,1.218304,0.000000
CAPHOTEL,5.863392e+09,3.865881,-0.022346
CHELLARAM,6.178800e+09,1.398106,-0.095238
CHIPLC,3.673755e+09,3.530753,0.000000
UPL,3.189134e+09,2.817793,-0.041667
STANBIC,2.727480e+09,3.399603,-0.076923
SCOAREDSTAREX,2.779050e+09,1.577222,-0.069068
REGALINS,2.934250e+09,2.596575,-0.063830
NEIMETH,3.323530e+09,1.802000,-0.054054
NAHCO,4.872660e+09,1.448108,0.276596


In [28]:
SL

,Market_Cap,Book_to_market_ratio,Change_in_Price
ABCTRANS,5.470410e+08,2.014109,-0.029412
ACADEMY,2.358720e+08,1.695623,0.000000
ALEX,1.583712e+09,1.306469,0.000000
CHAMPION,9.861726e+08,1.525179,0.000000
UPDCREIT,5.306343e+08,5.439075,0.016529
RTBRISCOE,5.650650e+08,6.855141,-0.395833
PHARMDEKO,5.097855e+08,1.835870,0.450617
NNFM,1.523610e+09,1.967623,0.171233
MULTIVERSE,8.523880e+08,1.026023,0.000000
MCNICHOLS,2.485000e+07,14.489697,0.000000


In [29]:
HML=(1/3*(SLPRICE+SMPRICE+SHPRICE)) - (1/3*(BLPRICE + BMPRICE + BHPRICE))

In [30]:
HML

-0.00022858947029031208

In [31]:
SMB=(1/2*(SHPRICE + BHPRICE)) - (1/2*(SLPRICE +BLPRICE ))

In [32]:
SMB

0.009392563344653115

In [33]:
price2=new_df.iloc[8,:].values.flatten().tolist()
october_2021=pd.DataFrame({'Market_Cap':[AIICO.iloc[1,26],FTNCOCOA.iloc[1,26],LIVINGTRUST.iloc[1,26],ABCTRANS.iloc[1,26],
                                           ACADEMY.iloc[1,26],AFRINSURE.iloc[1,26],AFROMEDIA.iloc[1,6],AIRTELAFRI.iloc[1,26],
                                           ALEX.iloc[1,26],BERGER.iloc[1,26],BETAGLAS.iloc[1,26],BUACEMENT.iloc[1,26],
                                           BUAFOODS.iloc[1,26],CADBURY.iloc[1,26],CAP.iloc[1,26],CAVERTON.iloc[1,26],
                                           CHAMPION.iloc[1,26],CHAMS.iloc[1,26],CHELLARAM.iloc[1,26],CHIPLC.iloc[1,26],
                                           CILEASING.iloc[1,26],ETRANZACT.iloc[1,26],CONOIL.iloc[1,26],TRANSCORP.iloc[1,26],
                                           UNILEVER.iloc[1,26],UPDCREIT.iloc[1,26],UPL.iloc[1,26],VERITASKAP.iloc[1,26],
                                           ZENITHBANK.iloc[1,26],WAPCO.iloc[1,26],UPDC.iloc[1,26],UBN.iloc[1,26],
                                           PZ.iloc[1,26],LIVESTOCK.iloc[1,26],JAIZBANK.iloc[1,26],FCMB.iloc[1,26],
                                           FIDELITYBK.iloc[1,26],UBA.iloc[1,26],TOTAL.iloc[1,26],STANBIC.iloc[1,26],
                                           SOVRENINS.iloc[1,26],RTBRISCOE.iloc[1,26],SCOA.iloc[1,26],REDSTAREX.iloc[1,6],
                                           REGALINS.iloc[1,6],PRESCO.iloc[1,6],PHARMDEKO.iloc[1,6],OANDO.iloc[1,6],
                                           OKOMUOIL.iloc[1,6],NNFM.iloc[1,6],NGXGROUP.iloc[1,6],NESTLE.iloc[1,6],
                                           NEIMETH.iloc[1,6],NB.iloc[1,6],MULTIVERSE.iloc[1,6],NAHCO.iloc[1,6],NASCON.iloc[1,6],
                                           MTNN.iloc[1,6],MAYBAKER.iloc[1,6],MCNICHOLS.iloc[1,6],LEARNAFRCA.iloc[1,6],
                                           LINKASSURE.iloc[1,6],INTBREW.iloc[1,6],GUINNESS.iloc[1,6],HONYFLOUR.iloc[1,6],
                                           GLAXOSMITH.iloc[1,6],FBNH.iloc[1,6],AFRIPRUD.iloc[1,6],COURTVILLE.iloc[1,6],
                                           CUSTODIAN.iloc[1,6],CWG.iloc[1,6],DANGCEM.iloc[1,6],ELLAHLAKES.iloc[1,6],
                                           ETERNA.iloc[1,6],ETI.iloc[1,6],FLOURMILL.iloc[1,6],LASACO.iloc[1,6],
                                           MANSARD.iloc[1,6],MBENEFIT.iloc[1,6],MEYER.iloc[1,6],MORISON.iloc[1,6],MRS.iloc[1,6],
                                           NEM.iloc[1,6],UACN.iloc[1,6],UCAP.iloc[1,6],UNIVINSURE.iloc[1,6],WAPIC.iloc[1,6],
                                           VITAFOAM.iloc[1,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[1,27],FTNCOCOA.iloc[1,27],LIVINGTRUST.iloc[1,27],
                                                     ABCTRANS.iloc[1,27],ACADEMY.iloc[1,27],AFRINSURE.iloc[1,27],
                                                     AFROMEDIA.iloc[1,7],AIRTELAFRI.iloc[1,27],ALEX.iloc[1,27],BERGER.iloc[1,27],
                                                     BETAGLAS.iloc[1,27],BUACEMENT.iloc[1,27],BUAFOODS.iloc[1,27],
                                                     CADBURY.iloc[1,27],CAP.iloc[1,27],CAVERTON.iloc[1,27],CHAMPION.iloc[1,27],
                                                     CHAMS.iloc[1,27],CHELLARAM.iloc[1,27],CHIPLC.iloc[1,27],CILEASING.iloc[1,27],
                                                     ETRANZACT.iloc[1,27],CONOIL.iloc[1,27],TRANSCORP.iloc[1,27],
                                                     UNILEVER.iloc[1,27],UPDCREIT.iloc[1,27],UPL.iloc[1,27],VERITASKAP.iloc[1,27],
                                                     ZENITHBANK.iloc[1,27],WAPCO.iloc[1,27],UPDC.iloc[1,27],UBN.iloc[1,27],
                                                     PZ.iloc[1,27],LIVESTOCK.iloc[1,27],JAIZBANK.iloc[1,27],FCMB.iloc[1,27],
                                                     FIDELITYBK.iloc[1,27],UBA.iloc[1,27],TOTAL.iloc[1,27],STANBIC.iloc[1,27],
                                                     SOVRENINS.iloc[1,27],RTBRISCOE.iloc[1,27],SCOA.iloc[1,27],
                                                     REDSTAREX.iloc[1,7],
                                                     REGALINS.iloc[1,7],PRESCO.iloc[1,7],PHARMDEKO.iloc[1,7],OANDO.iloc[1,7],
                                                     OKOMUOIL.iloc[1,7],NNFM.iloc[1,7],NGXGROUP.iloc[1,7],NESTLE.iloc[1,7],
                                                     NEIMETH.iloc[1,7],NB.iloc[1,7],MULTIVERSE.iloc[1,7],NAHCO.iloc[1,7],
                                                     NASCON.iloc[1,7],MTNN.iloc[1,7],MAYBAKER.iloc[1,7],MCNICHOLS.iloc[1,7],
                                                     LEARNAFRCA.iloc[1,7],LINKASSURE.iloc[1,7],INTBREW.iloc[1,7],
                                                     GUINNESS.iloc[1,7],HONYFLOUR.iloc[1,7],GLAXOSMITH.iloc[1,7],FBNH.iloc[1,7],
                                                     AFRIPRUD.iloc[1,7],COURTVILLE.iloc[1,7],CUSTODIAN.iloc[1,7],CWG.iloc[1,7],
                                                     DANGCEM.iloc[1,7],ELLAHLAKES.iloc[1,7],ETERNA.iloc[1,7],ETI.iloc[1,7],
                                                     FLOURMILL.iloc[1,7],LASACO.iloc[1,7],MANSARD.iloc[1,7],MBENEFIT.iloc[1,7],
                                                     MEYER.iloc[1,7],MORISON.iloc[1,7],MRS.iloc[1,7],NEM.iloc[1,7],
                                                     UACN.iloc[1,7],UCAP.iloc[1,7],UNIVINSURE.iloc[1,7],WAPIC.iloc[1,7],
                                                     VITAFOAM.iloc[1,7]],
                             
                            
                            'Change_in_Price':price2},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [34]:
october_2021

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,1.945312e+10,1.840655,0.305265
FTNCOCOA,9.680000e+08,-0.125093,-0.102041
LIVINGTRUST,4.000000e+09,0.681795,0.194030
ABCTRANS,5.138870e+08,2.144051,-0.060606
ACADEMY,1.995840e+08,2.003918,-0.153846
...,...,...,...
UACN,3.255869e+10,-0.876743,0.102439
UCAP,5.520000e+10,0.487120,0.027933
UNIVINSURE,3.520000e+09,2.639955,0.100000
WAPIC,1.151606e+10,2.000163,0.000000


In [35]:
SL1=october_2021[(october_2021['Market_Cap']<=2000000000)&(october_2021['Book_to_market_ratio']>1)]

SM1=october_2021[(october_2021['Market_Cap']>2000000000) & (october_2021['Market_Cap']<10000000000) & (october_2021['Book_to_market_ratio']>1)]

SH1=october_2021[(october_2021['Market_Cap']>=10000000000)&(october_2021['Book_to_market_ratio']>1)]

BL1=october_2021[(october_2021['Market_Cap']<=2000000000) & (october_2021['Book_to_market_ratio']<1)]

BH1=october_2021[(october_2021['Market_Cap']>=10000000000)&(october_2021['Book_to_market_ratio']<1)]

BM1=october_2021[(october_2021['Market_Cap']>2000000000) & (october_2021['Market_Cap']<10000000000) & (october_2021['Book_to_market_ratio']<1)]


In [36]:
SLPRICE1=SL1['Change_in_Price'].mean()

SMPRICE1=SM1['Change_in_Price'].mean()

SHPRICE1=SH1['Change_in_Price'].mean()

BLPRICE1=BL1['Change_in_Price'].mean()

BMPRICE1=BM1['Change_in_Price'].mean()

BHPRICE1=BH1['Change_in_Price'].mean()


In [37]:
HML1=(1/3*(SLPRICE1+SMPRICE1+SHPRICE1)) - (1/3*(BLPRICE1 + BMPRICE1 + BHPRICE1))

In [38]:

SMB1=(1/2*(SHPRICE1 + BHPRICE1)) - (1/2*(SLPRICE1 +BLPRICE1 ))

In [39]:
HML1,SMB1

(0.039417193128587026, 0.08844091403335673)

In [40]:
price3=new_df.iloc[9,:].values.flatten().tolist()
november_2021=pd.DataFrame({'Market_Cap':[AIICO.iloc[2,26],FTNCOCOA.iloc[2,26],LIVINGTRUST.iloc[2,26],ABCTRANS.iloc[2,26],
                                           ACADEMY.iloc[2,26],AFRINSURE.iloc[2,26],AFROMEDIA.iloc[2,6],AIRTELAFRI.iloc[2,26],
                                           ALEX.iloc[2,26],BERGER.iloc[2,26],BETAGLAS.iloc[2,26],BUACEMENT.iloc[2,26],
                                           BUAFOODS.iloc[2,26],CADBURY.iloc[2,26],CAP.iloc[2,26],CAVERTON.iloc[2,26],
                                           CHAMPION.iloc[2,26],CHAMS.iloc[2,26],CHELLARAM.iloc[2,26],CHIPLC.iloc[2,26],
                                           CILEASING.iloc[2,26],ETRANZACT.iloc[2,26],CONOIL.iloc[2,26],TRANSCORP.iloc[2,26],
                                           UNILEVER.iloc[2,26],UPDCREIT.iloc[2,26],UPL.iloc[2,26],VERITASKAP.iloc[2,26],
                                           ZENITHBANK.iloc[2,26],WAPCO.iloc[2,26],UPDC.iloc[2,26],UBN.iloc[2,26],
                                           PZ.iloc[2,26],LIVESTOCK.iloc[2,26],JAIZBANK.iloc[2,26],FCMB.iloc[2,26],
                                           FIDELITYBK.iloc[2,26],UBA.iloc[2,26],TOTAL.iloc[2,26],STANBIC.iloc[2,26],
                                           SOVRENINS.iloc[2,26],RTBRISCOE.iloc[2,26],SCOA.iloc[2,26],REDSTAREX.iloc[2,6],
                                           REGALINS.iloc[2,6],PRESCO.iloc[2,6],PHARMDEKO.iloc[2,6],OANDO.iloc[2,6],
                                           OKOMUOIL.iloc[2,6],NNFM.iloc[2,6],NGXGROUP.iloc[2,6],NESTLE.iloc[2,6],
                                           NEIMETH.iloc[2,6],NB.iloc[2,6],MULTIVERSE.iloc[2,6],NAHCO.iloc[2,6],NASCON.iloc[2,6],
                                           MTNN.iloc[2,6],MAYBAKER.iloc[2,6],MCNICHOLS.iloc[2,6],LEARNAFRCA.iloc[2,6],
                                           LINKASSURE.iloc[2,6],INTBREW.iloc[2,6],GUINNESS.iloc[2,6],HONYFLOUR.iloc[2,6],
                                           GLAXOSMITH.iloc[2,6],FBNH.iloc[2,6],AFRIPRUD.iloc[2,6],COURTVILLE.iloc[2,6],
                                           CUSTODIAN.iloc[2,6],CWG.iloc[2,6],DANGCEM.iloc[2,6],ELLAHLAKES.iloc[2,6],
                                           ETERNA.iloc[2,6],ETI.iloc[2,6],FLOURMILL.iloc[2,6],LASACO.iloc[2,6],
                                           MANSARD.iloc[2,6],MBENEFIT.iloc[2,6],MEYER.iloc[2,6],MORISON.iloc[2,6],MRS.iloc[2,6],
                                           NEM.iloc[2,6],UACN.iloc[2,6],UCAP.iloc[2,6],UNIVINSURE.iloc[2,6],WAPIC.iloc[2,6],
                                           VITAFOAM.iloc[2,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[2,27],FTNCOCOA.iloc[2,27],LIVINGTRUST.iloc[2,27],
                                                     ABCTRANS.iloc[2,27],ACADEMY.iloc[2,27],AFRINSURE.iloc[2,27],
                                                     AFROMEDIA.iloc[2,7],AIRTELAFRI.iloc[2,27],ALEX.iloc[2,27],BERGER.iloc[2,27],
                                                     BETAGLAS.iloc[2,27],BUACEMENT.iloc[2,27],BUAFOODS.iloc[2,27],
                                                     CADBURY.iloc[2,27],CAP.iloc[2,27],CAVERTON.iloc[2,27],CHAMPION.iloc[2,27],
                                                     CHAMS.iloc[2,27],CHELLARAM.iloc[2,27],CHIPLC.iloc[2,27],CILEASING.iloc[2,27],
                                                     ETRANZACT.iloc[2,27],CONOIL.iloc[2,27],TRANSCORP.iloc[2,27],
                                                     UNILEVER.iloc[2,27],UPDCREIT.iloc[2,27],UPL.iloc[2,27],VERITASKAP.iloc[2,27],
                                                     ZENITHBANK.iloc[2,27],WAPCO.iloc[2,27],UPDC.iloc[2,27],UBN.iloc[2,27],
                                                     PZ.iloc[2,27],LIVESTOCK.iloc[2,27],JAIZBANK.iloc[2,27],FCMB.iloc[2,27],
                                                     FIDELITYBK.iloc[2,27],UBA.iloc[2,27],TOTAL.iloc[2,27],STANBIC.iloc[2,27],
                                                     SOVRENINS.iloc[2,27],RTBRISCOE.iloc[2,27],SCOA.iloc[2,27],
                                                     REDSTAREX.iloc[2,7],
                                                     REGALINS.iloc[2,7],PRESCO.iloc[2,7],PHARMDEKO.iloc[2,7],OANDO.iloc[2,7],
                                                     OKOMUOIL.iloc[2,7],NNFM.iloc[2,7],NGXGROUP.iloc[2,7],NESTLE.iloc[2,7],
                                                     NEIMETH.iloc[2,7],NB.iloc[2,7],MULTIVERSE.iloc[2,7],NAHCO.iloc[2,7],
                                                     NASCON.iloc[2,7],MTNN.iloc[2,7],MAYBAKER.iloc[2,7],MCNICHOLS.iloc[2,7],
                                                     LEARNAFRCA.iloc[2,7],LINKASSURE.iloc[2,7],INTBREW.iloc[2,7],
                                                     GUINNESS.iloc[2,7],HONYFLOUR.iloc[2,7],GLAXOSMITH.iloc[2,7],FBNH.iloc[2,7],
                                                     AFRIPRUD.iloc[2,7],COURTVILLE.iloc[2,7],CUSTODIAN.iloc[2,7],CWG.iloc[2,7],
                                                     DANGCEM.iloc[2,7],ELLAHLAKES.iloc[2,7],ETERNA.iloc[2,7],ETI.iloc[2,7],
                                                     FLOURMILL.iloc[2,7],LASACO.iloc[2,7],MANSARD.iloc[2,7],MBENEFIT.iloc[2,7],
                                                     MEYER.iloc[2,7],MORISON.iloc[2,7],MRS.iloc[2,7],NEM.iloc[2,7],
                                                     UACN.iloc[2,7],UCAP.iloc[2,7],UNIVINSURE.iloc[2,7],WAPIC.iloc[2,7],
                                                     VITAFOAM.iloc[2,7]],
                             
                            
                            'Change_in_Price':price3},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [41]:
november_2021

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,3.038247e+10,1.178525,0.561830
FTNCOCOA,8.580000e+08,-0.141131,-0.113636
LIVINGTRUST,4.800000e+09,0.568163,0.200000
ABCTRANS,5.636180e+08,1.954870,0.096774
ACADEMY,2.177280e+08,1.836925,0.090909
...,...,...,...
UACN,2.881300e+10,-0.990719,-0.115044
UCAP,5.790000e+10,0.464405,0.048913
UNIVINSURE,3.200000e+09,2.903950,-0.090909
WAPIC,1.199590e+10,1.920156,0.041667


In [42]:
SL2=november_2021[(november_2021['Market_Cap']<=2000000000)&(november_2021['Book_to_market_ratio']>1)]

SM2=november_2021[(november_2021['Market_Cap']>2000000000) & (november_2021['Market_Cap']<10000000000) & (november_2021['Book_to_market_ratio']>1)]

SH2=november_2021[(november_2021['Market_Cap']>=10000000000)&(november_2021['Book_to_market_ratio']>1)]

BL2=november_2021[(november_2021['Market_Cap']<=2000000000) & (november_2021['Book_to_market_ratio']<1)]

BH2=november_2021[(november_2021['Market_Cap']>=10000000000)&(november_2021['Book_to_market_ratio']<1)]

BM2=november_2021[(november_2021['Market_Cap']>2000000000) & (november_2021['Market_Cap']<10000000000) & (november_2021['Book_to_market_ratio']<1)]


In [43]:
SLPRICE2=SL2['Change_in_Price'].mean()

SMPRICE2=SM2['Change_in_Price'].mean()

SHPRICE2=SH2['Change_in_Price'].mean()

BLPRICE2=BL2['Change_in_Price'].mean()

BMPRICE2=BM2['Change_in_Price'].mean()

BHPRICE2=BH2['Change_in_Price'].mean()



In [44]:
HML2=(1/3*(SLPRICE2+SMPRICE2+SHPRICE2)) - (1/3*(BLPRICE2 + BMPRICE2 + BHPRICE2))

SMB2=(1/2*(SHPRICE2 + BHPRICE2)) - (1/2*(SLPRICE2 +BLPRICE2))

HML2,SMB2

(-0.011932074493838148, -0.01711445192426752)

In [45]:
price4=new_df.iloc[10,:].values.flatten().tolist()

december_2021=pd.DataFrame({'Market_Cap':[AIICO.iloc[3,26],FTNCOCOA.iloc[3,26],LIVINGTRUST.iloc[3,26],ABCTRANS.iloc[3,26],
                                           ACADEMY.iloc[3,26],AFRINSURE.iloc[3,26],AFROMEDIA.iloc[3,6],AIRTELAFRI.iloc[3,26],
                                           ALEX.iloc[3,26],BERGER.iloc[3,26],BETAGLAS.iloc[3,26],BUACEMENT.iloc[3,26],
                                           BUAFOODS.iloc[3,26],CADBURY.iloc[3,26],CAP.iloc[3,26],CAVERTON.iloc[3,26],
                                           CHAMPION.iloc[3,26],CHAMS.iloc[3,26],CHELLARAM.iloc[3,26],CHIPLC.iloc[3,26],
                                           CILEASING.iloc[3,26],ETRANZACT.iloc[3,26],CONOIL.iloc[3,26],TRANSCORP.iloc[3,26],
                                           UNILEVER.iloc[3,26],UPDCREIT.iloc[3,26],UPL.iloc[3,26],VERITASKAP.iloc[3,26],
                                           ZENITHBANK.iloc[3,26],WAPCO.iloc[3,26],UPDC.iloc[3,26],UBN.iloc[3,26],
                                           PZ.iloc[3,26],LIVESTOCK.iloc[3,26],JAIZBANK.iloc[3,26],FCMB.iloc[3,26],
                                           FIDELITYBK.iloc[3,26],UBA.iloc[3,26],TOTAL.iloc[3,26],STANBIC.iloc[3,26],
                                           SOVRENINS.iloc[3,26],RTBRISCOE.iloc[3,26],SCOA.iloc[3,26],REDSTAREX.iloc[3,6],
                                           REGALINS.iloc[3,6],PRESCO.iloc[3,6],PHARMDEKO.iloc[3,6],OANDO.iloc[3,6],
                                           OKOMUOIL.iloc[3,6],NNFM.iloc[3,6],NGXGROUP.iloc[3,6],NESTLE.iloc[3,6],
                                           NEIMETH.iloc[3,6],NB.iloc[3,6],MULTIVERSE.iloc[3,6],NAHCO.iloc[3,6],NASCON.iloc[3,6],
                                           MTNN.iloc[3,6],MAYBAKER.iloc[3,6],MCNICHOLS.iloc[3,6],LEARNAFRCA.iloc[3,6],
                                           LINKASSURE.iloc[3,6],INTBREW.iloc[3,6],GUINNESS.iloc[3,6],HONYFLOUR.iloc[3,6],
                                           GLAXOSMITH.iloc[3,6],FBNH.iloc[3,6],AFRIPRUD.iloc[3,6],COURTVILLE.iloc[3,6],
                                           CUSTODIAN.iloc[3,6],CWG.iloc[3,6],DANGCEM.iloc[3,6],ELLAHLAKES.iloc[3,6],
                                           ETERNA.iloc[3,6],ETI.iloc[3,6],FLOURMILL.iloc[3,6],LASACO.iloc[3,6],
                                           MANSARD.iloc[3,6],MBENEFIT.iloc[3,6],MEYER.iloc[3,6],MORISON.iloc[3,6],MRS.iloc[3,6],
                                           NEM.iloc[3,6],UACN.iloc[3,6],UCAP.iloc[3,6],UNIVINSURE.iloc[3,6],WAPIC.iloc[3,6],
                                           VITAFOAM.iloc[3,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[3,27],FTNCOCOA.iloc[3,27],LIVINGTRUST.iloc[3,27],
                                                     ABCTRANS.iloc[3,27],ACADEMY.iloc[3,27],AFRINSURE.iloc[3,27],
                                                     AFROMEDIA.iloc[3,7],AIRTELAFRI.iloc[3,27],ALEX.iloc[3,27],BERGER.iloc[3,27],
                                                     BETAGLAS.iloc[3,27],BUACEMENT.iloc[3,27],BUAFOODS.iloc[3,27],
                                                     CADBURY.iloc[3,27],CAP.iloc[3,27],CAVERTON.iloc[3,27],CHAMPION.iloc[3,27],
                                                     CHAMS.iloc[3,27],CHELLARAM.iloc[3,27],CHIPLC.iloc[3,27],CILEASING.iloc[3,27],
                                                     ETRANZACT.iloc[3,27],CONOIL.iloc[3,27],TRANSCORP.iloc[3,27],
                                                     UNILEVER.iloc[3,27],UPDCREIT.iloc[3,27],UPL.iloc[3,27],VERITASKAP.iloc[3,27],
                                                     ZENITHBANK.iloc[3,27],WAPCO.iloc[3,27],UPDC.iloc[3,27],UBN.iloc[3,27],
                                                     PZ.iloc[3,27],LIVESTOCK.iloc[3,27],JAIZBANK.iloc[3,27],FCMB.iloc[3,27],
                                                     FIDELITYBK.iloc[3,27],UBA.iloc[3,27],TOTAL.iloc[3,27],STANBIC.iloc[3,27],
                                                     SOVRENINS.iloc[3,27],RTBRISCOE.iloc[3,27],SCOA.iloc[3,27],
                                                     REDSTAREX.iloc[3,7],
                                                     REGALINS.iloc[3,7],PRESCO.iloc[3,7],PHARMDEKO.iloc[3,7],OANDO.iloc[3,7],
                                                     OKOMUOIL.iloc[3,7],NNFM.iloc[3,7],NGXGROUP.iloc[3,7],NESTLE.iloc[3,7],
                                                     NEIMETH.iloc[3,7],NB.iloc[3,7],MULTIVERSE.iloc[3,7],NAHCO.iloc[3,7],
                                                     NASCON.iloc[3,7],MTNN.iloc[3,7],MAYBAKER.iloc[3,7],MCNICHOLS.iloc[3,7],
                                                     LEARNAFRCA.iloc[3,7],LINKASSURE.iloc[3,7],INTBREW.iloc[3,7],
                                                     GUINNESS.iloc[3,7],HONYFLOUR.iloc[3,7],GLAXOSMITH.iloc[3,7],FBNH.iloc[3,7],
                                                     AFRIPRUD.iloc[3,7],COURTVILLE.iloc[3,7],CUSTODIAN.iloc[3,7],CWG.iloc[3,7],
                                                     DANGCEM.iloc[3,7],ELLAHLAKES.iloc[3,7],ETERNA.iloc[3,7],ETI.iloc[3,7],
                                                     FLOURMILL.iloc[3,7],LASACO.iloc[3,7],MANSARD.iloc[3,7],MBENEFIT.iloc[3,7],
                                                     MEYER.iloc[3,7],MORISON.iloc[3,7],MRS.iloc[3,7],NEM.iloc[3,7],
                                                     UACN.iloc[3,7],UCAP.iloc[3,7],UNIVINSURE.iloc[3,7],WAPIC.iloc[3,7],
                                                     VITAFOAM.iloc[3,7]],
                             
                            
                            'Change_in_Price':price4},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [46]:
december_2021

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,2.562370e+10,1.484132,-0.156627
FTNCOCOA,8.580000e+08,-0.875991,0.000000
LIVINGTRUST,5.200000e+09,0.620031,0.083333
ABCTRANS,5.138870e+08,1.254264,-0.088235
ACADEMY,3.024000e+08,0.954762,0.388889
...,...,...,...
UACN,2.737235e+10,1.857276,-0.050000
UCAP,5.940000e+10,0.514256,0.025907
UNIVINSURE,3.200000e+09,2.923191,0.000000
WAPIC,1.343541e+10,1.675647,0.120000


In [47]:
SL3=december_2021[(december_2021['Market_Cap']<=2000000000)&(december_2021['Book_to_market_ratio']>1)]

SM3=december_2021[(december_2021['Market_Cap']>2000000000) & (december_2021['Market_Cap']<10000000000) & (december_2021['Book_to_market_ratio']>1)]

SH3=december_2021[(december_2021['Market_Cap']>=10000000000)&(december_2021['Book_to_market_ratio']>1)]

BL3=december_2021[(december_2021['Market_Cap']<=2000000000) & (december_2021['Book_to_market_ratio']<1)]

BH3=december_2021[(december_2021['Market_Cap']>=10000000000)&(december_2021['Book_to_market_ratio']<1)]

BM3=december_2021[(december_2021['Market_Cap']>2000000000) & (december_2021['Market_Cap']<10000000000) & (december_2021['Book_to_market_ratio']<1)]


In [48]:
SLPRICE3=SL3['Change_in_Price'].mean()

SMPRICE3=SM3['Change_in_Price'].mean()

SHPRICE3=SH3['Change_in_Price'].mean()

BLPRICE3=BL3['Change_in_Price'].mean()

BMPRICE3=BM3['Change_in_Price'].mean()

BHPRICE3=BH3['Change_in_Price'].mean()



In [49]:
HML3=(1/3*(SLPRICE3+SMPRICE3+SHPRICE3)) - (1/3*(BLPRICE3 + BMPRICE3 + BHPRICE3))

SMB3=(1/2*(SHPRICE3 + BHPRICE3)) - (1/2*(SLPRICE3 +BLPRICE3))

HML3,SMB3

(0.01885888583857296, -0.07509946881305019)

In [50]:
price5=new_df.iloc[11,:].values.flatten().tolist()

january_2022=pd.DataFrame({'Market_Cap':[AIICO.iloc[4,26],FTNCOCOA.iloc[4,26],LIVINGTRUST.iloc[4,26],ABCTRANS.iloc[4,26],
                                           ACADEMY.iloc[4,26],AFRINSURE.iloc[4,26],AFROMEDIA.iloc[4,6],AIRTELAFRI.iloc[4,26],
                                           ALEX.iloc[4,26],BERGER.iloc[4,26],BETAGLAS.iloc[4,26],BUACEMENT.iloc[4,26],
                                           BUAFOODS.iloc[4,26],CADBURY.iloc[4,26],CAP.iloc[4,26],CAVERTON.iloc[4,26],
                                           CHAMPION.iloc[4,26],CHAMS.iloc[4,26],CHELLARAM.iloc[4,26],CHIPLC.iloc[4,26],
                                           CILEASING.iloc[4,26],ETRANZACT.iloc[4,26],CONOIL.iloc[4,26],TRANSCORP.iloc[4,26],
                                           UNILEVER.iloc[4,26],UPDCREIT.iloc[4,26],UPL.iloc[4,26],VERITASKAP.iloc[4,26],
                                           ZENITHBANK.iloc[4,26],WAPCO.iloc[4,26],UPDC.iloc[4,26],UBN.iloc[4,26],
                                           PZ.iloc[4,26],LIVESTOCK.iloc[4,26],JAIZBANK.iloc[4,26],FCMB.iloc[4,26],
                                           FIDELITYBK.iloc[4,26],UBA.iloc[4,26],TOTAL.iloc[4,26],STANBIC.iloc[4,26],
                                           SOVRENINS.iloc[4,26],RTBRISCOE.iloc[4,26],SCOA.iloc[4,26],REDSTAREX.iloc[4,6],
                                           REGALINS.iloc[4,6],PRESCO.iloc[4,6],PHARMDEKO.iloc[4,6],OANDO.iloc[4,6],
                                           OKOMUOIL.iloc[4,6],NNFM.iloc[4,6],NGXGROUP.iloc[4,6],NESTLE.iloc[4,6],
                                           NEIMETH.iloc[4,6],NB.iloc[4,6],MULTIVERSE.iloc[4,6],NAHCO.iloc[4,6],NASCON.iloc[4,6],
                                           MTNN.iloc[4,6],MAYBAKER.iloc[4,6],MCNICHOLS.iloc[4,6],LEARNAFRCA.iloc[4,6],
                                           LINKASSURE.iloc[4,6],INTBREW.iloc[4,6],GUINNESS.iloc[4,6],HONYFLOUR.iloc[4,6],
                                           GLAXOSMITH.iloc[4,6],FBNH.iloc[4,6],AFRIPRUD.iloc[4,6],COURTVILLE.iloc[4,6],
                                           CUSTODIAN.iloc[4,6],CWG.iloc[4,6],DANGCEM.iloc[4,6],ELLAHLAKES.iloc[4,6],
                                           ETERNA.iloc[4,6],ETI.iloc[4,6],FLOURMILL.iloc[4,6],LASACO.iloc[4,6],
                                           MANSARD.iloc[4,6],MBENEFIT.iloc[4,6],MEYER.iloc[4,6],MORISON.iloc[4,6],MRS.iloc[4,6],
                                           NEM.iloc[4,6],UACN.iloc[4,6],UCAP.iloc[4,6],UNIVINSURE.iloc[4,6],WAPIC.iloc[4,6],
                                           VITAFOAM.iloc[4,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[4,27],FTNCOCOA.iloc[4,27],LIVINGTRUST.iloc[4,27],
                                                     ABCTRANS.iloc[4,27],ACADEMY.iloc[4,27],AFRINSURE.iloc[4,27],
                                                     AFROMEDIA.iloc[4,7],AIRTELAFRI.iloc[4,27],ALEX.iloc[4,27],BERGER.iloc[4,27],
                                                     BETAGLAS.iloc[4,27],BUACEMENT.iloc[4,27],BUAFOODS.iloc[4,27],
                                                     CADBURY.iloc[4,27],CAP.iloc[4,27],CAVERTON.iloc[4,27],CHAMPION.iloc[4,27],
                                                     CHAMS.iloc[4,27],CHELLARAM.iloc[4,27],CHIPLC.iloc[4,27],CILEASING.iloc[4,27],
                                                     ETRANZACT.iloc[4,27],CONOIL.iloc[4,27],TRANSCORP.iloc[4,27],
                                                     UNILEVER.iloc[4,27],UPDCREIT.iloc[4,27],UPL.iloc[4,27],VERITASKAP.iloc[4,27],
                                                     ZENITHBANK.iloc[4,27],WAPCO.iloc[4,27],UPDC.iloc[4,27],UBN.iloc[4,27],
                                                     PZ.iloc[4,27],LIVESTOCK.iloc[4,27],JAIZBANK.iloc[4,27],FCMB.iloc[4,27],
                                                     FIDELITYBK.iloc[4,27],UBA.iloc[4,27],TOTAL.iloc[4,27],STANBIC.iloc[4,27],
                                                     SOVRENINS.iloc[4,27],RTBRISCOE.iloc[4,27],SCOA.iloc[4,27],
                                                     REDSTAREX.iloc[4,7],
                                                     REGALINS.iloc[4,7],PRESCO.iloc[4,7],PHARMDEKO.iloc[4,7],OANDO.iloc[4,7],
                                                     OKOMUOIL.iloc[4,7],NNFM.iloc[4,7],NGXGROUP.iloc[4,7],NESTLE.iloc[4,7],
                                                     NEIMETH.iloc[4,7],NB.iloc[4,7],MULTIVERSE.iloc[4,7],NAHCO.iloc[4,7],
                                                     NASCON.iloc[4,7],MTNN.iloc[4,7],MAYBAKER.iloc[4,7],MCNICHOLS.iloc[4,7],
                                                     LEARNAFRCA.iloc[4,7],LINKASSURE.iloc[4,7],INTBREW.iloc[4,7],
                                                     GUINNESS.iloc[4,7],HONYFLOUR.iloc[4,7],GLAXOSMITH.iloc[4,7],FBNH.iloc[4,7],
                                                     AFRIPRUD.iloc[4,7],COURTVILLE.iloc[4,7],CUSTODIAN.iloc[4,7],CWG.iloc[4,7],
                                                     DANGCEM.iloc[4,7],ELLAHLAKES.iloc[4,7],ETERNA.iloc[4,7],ETI.iloc[4,7],
                                                     FLOURMILL.iloc[4,7],LASACO.iloc[4,7],MANSARD.iloc[4,7],MBENEFIT.iloc[4,7],
                                                     MEYER.iloc[4,7],MORISON.iloc[4,7],MRS.iloc[4,7],NEM.iloc[4,7],
                                                     UACN.iloc[4,7],UCAP.iloc[4,7],UNIVINSURE.iloc[4,7],WAPIC.iloc[4,7],
                                                     VITAFOAM.iloc[4,7]],
                             
                            
                            'Change_in_Price':price5},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [51]:
january_2022

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,2.745396e+10,1.385190,0.071429
FTNCOCOA,7.700000e+08,-0.976104,-0.102564
LIVINGTRUST,5.700000e+09,0.565642,0.096154
ABCTRANS,5.967720e+08,1.080061,0.161290
ACADEMY,5.685120e+08,0.507852,0.880000
...,...,...,...
UACN,2.564357e+10,1.982485,-0.063158
UCAP,6.600000e+10,0.462830,0.111111
UNIVINSURE,3.200000e+09,2.923191,0.000000
WAPIC,1.199590e+10,1.876725,-0.107143


In [52]:
SL4=january_2022[(january_2022['Market_Cap']<=2000000000)&(january_2022['Book_to_market_ratio']>1)]

SM4=january_2022[(january_2022['Market_Cap']>2000000000) & (january_2022['Market_Cap']<10000000000) & (january_2022['Book_to_market_ratio']>1)]

SH4=january_2022[(january_2022['Market_Cap']>=10000000000)&(january_2022['Book_to_market_ratio']>1)]

BL4=january_2022[(january_2022['Market_Cap']<=2000000000) & (january_2022['Book_to_market_ratio']<1)]

BH4=january_2022[(january_2022['Market_Cap']>=10000000000)&(january_2022['Book_to_market_ratio']<1)]

BM4=january_2022[(january_2022['Market_Cap']>2000000000) & (january_2022['Market_Cap']<10000000000) & (january_2022['Book_to_market_ratio']<1)]


In [53]:
SLPRICE4=SL4['Change_in_Price'].mean()

SMPRICE4=SM4['Change_in_Price'].mean()

SHPRICE4=SH4['Change_in_Price'].mean()

BLPRICE4=BL4['Change_in_Price'].mean()

BMPRICE4=BM4['Change_in_Price'].mean()

BHPRICE4=BH4['Change_in_Price'].mean()



In [54]:
HML4=(1/3*(SLPRICE4+SMPRICE4+SHPRICE4)) - (1/3*(BLPRICE4 + BMPRICE4 + BHPRICE4))

SMB4=(1/2*(SHPRICE4 + BHPRICE4)) - (1/2*(SLPRICE4 +BLPRICE4))

HML4,SMB4

(-0.05413207140131848, -0.0342124048891446)

In [55]:
price6=new_df.iloc[12,:].values.flatten().tolist()

february_2022=pd.DataFrame({'Market_Cap':[AIICO.iloc[5,26],FTNCOCOA.iloc[5,26],LIVINGTRUST.iloc[5,26],ABCTRANS.iloc[5,26],
                                           ACADEMY.iloc[5,26],AFRINSURE.iloc[5,26],AFROMEDIA.iloc[5,6],AIRTELAFRI.iloc[5,26],
                                           ALEX.iloc[5,26],BERGER.iloc[5,26],BETAGLAS.iloc[5,26],BUACEMENT.iloc[5,26],
                                           BUAFOODS.iloc[5,26],CADBURY.iloc[5,26],CAP.iloc[5,26],CAVERTON.iloc[5,26],
                                           CHAMPION.iloc[5,26],CHAMS.iloc[5,26],CHELLARAM.iloc[5,26],CHIPLC.iloc[5,26],
                                           CILEASING.iloc[5,26],ETRANZACT.iloc[5,26],CONOIL.iloc[5,26],TRANSCORP.iloc[5,26],
                                           UNILEVER.iloc[5,26],UPDCREIT.iloc[5,26],UPL.iloc[5,26],VERITASKAP.iloc[5,26],
                                           ZENITHBANK.iloc[5,26],WAPCO.iloc[5,26],UPDC.iloc[5,26],UBN.iloc[5,26],
                                           PZ.iloc[5,26],LIVESTOCK.iloc[5,26],JAIZBANK.iloc[5,26],FCMB.iloc[5,26],
                                           FIDELITYBK.iloc[5,26],UBA.iloc[5,26],TOTAL.iloc[5,26],STANBIC.iloc[5,26],
                                           SOVRENINS.iloc[5,26],RTBRISCOE.iloc[5,26],SCOA.iloc[5,26],REDSTAREX.iloc[5,6],
                                           REGALINS.iloc[5,6],PRESCO.iloc[5,6],PHARMDEKO.iloc[5,6],OANDO.iloc[5,6],
                                           OKOMUOIL.iloc[5,6],NNFM.iloc[5,6],NGXGROUP.iloc[5,6],NESTLE.iloc[5,6],
                                           NEIMETH.iloc[5,6],NB.iloc[5,6],MULTIVERSE.iloc[5,6],NAHCO.iloc[5,6],NASCON.iloc[5,6],
                                           MTNN.iloc[5,6],MAYBAKER.iloc[5,6],MCNICHOLS.iloc[5,6],LEARNAFRCA.iloc[5,6],
                                           LINKASSURE.iloc[5,6],INTBREW.iloc[5,6],GUINNESS.iloc[5,6],HONYFLOUR.iloc[5,6],
                                           GLAXOSMITH.iloc[5,6],FBNH.iloc[5,6],AFRIPRUD.iloc[5,6],COURTVILLE.iloc[5,6],
                                           CUSTODIAN.iloc[5,6],CWG.iloc[5,6],DANGCEM.iloc[5,6],ELLAHLAKES.iloc[5,6],
                                           ETERNA.iloc[5,6],ETI.iloc[5,6],FLOURMILL.iloc[5,6],LASACO.iloc[5,6],
                                           MANSARD.iloc[5,6],MBENEFIT.iloc[5,6],MEYER.iloc[5,6],MORISON.iloc[5,6],MRS.iloc[5,6],
                                           NEM.iloc[5,6],UACN.iloc[5,6],UCAP.iloc[5,6],UNIVINSURE.iloc[5,6],WAPIC.iloc[5,6],
                                           VITAFOAM.iloc[5,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[5,27],FTNCOCOA.iloc[5,27],LIVINGTRUST.iloc[5,27],
                                                     ABCTRANS.iloc[5,27],ACADEMY.iloc[5,27],AFRINSURE.iloc[5,27],
                                                     AFROMEDIA.iloc[5,7],AIRTELAFRI.iloc[5,27],ALEX.iloc[5,27],BERGER.iloc[5,27],
                                                     BETAGLAS.iloc[5,27],BUACEMENT.iloc[5,27],BUAFOODS.iloc[5,27],
                                                     CADBURY.iloc[5,27],CAP.iloc[5,27],CAVERTON.iloc[5,27],CHAMPION.iloc[5,27],
                                                     CHAMS.iloc[5,27],CHELLARAM.iloc[5,27],CHIPLC.iloc[5,27],CILEASING.iloc[5,27],
                                                     ETRANZACT.iloc[5,27],CONOIL.iloc[5,27],TRANSCORP.iloc[5,27],
                                                     UNILEVER.iloc[5,27],UPDCREIT.iloc[5,27],UPL.iloc[5,27],VERITASKAP.iloc[5,27],
                                                     ZENITHBANK.iloc[5,27],WAPCO.iloc[5,27],UPDC.iloc[5,27],UBN.iloc[5,27],
                                                     PZ.iloc[5,27],LIVESTOCK.iloc[5,27],JAIZBANK.iloc[5,27],FCMB.iloc[5,27],
                                                     FIDELITYBK.iloc[5,27],UBA.iloc[5,27],TOTAL.iloc[5,27],STANBIC.iloc[5,27],
                                                     SOVRENINS.iloc[5,27],RTBRISCOE.iloc[5,27],SCOA.iloc[5,27],
                                                     REDSTAREX.iloc[5,7],
                                                     REGALINS.iloc[5,7],PRESCO.iloc[5,7],PHARMDEKO.iloc[5,7],OANDO.iloc[5,7],
                                                     OKOMUOIL.iloc[5,7],NNFM.iloc[5,7],NGXGROUP.iloc[5,7],NESTLE.iloc[5,7],
                                                     NEIMETH.iloc[5,7],NB.iloc[5,7],MULTIVERSE.iloc[5,7],NAHCO.iloc[5,7],
                                                     NASCON.iloc[5,7],MTNN.iloc[5,7],MAYBAKER.iloc[5,7],MCNICHOLS.iloc[5,7],
                                                     LEARNAFRCA.iloc[5,7],LINKASSURE.iloc[5,7],INTBREW.iloc[5,7],
                                                     GUINNESS.iloc[5,7],HONYFLOUR.iloc[5,7],GLAXOSMITH.iloc[5,7],FBNH.iloc[5,7],
                                                     AFRIPRUD.iloc[5,7],COURTVILLE.iloc[5,7],CUSTODIAN.iloc[5,7],CWG.iloc[5,7],
                                                     DANGCEM.iloc[5,7],ELLAHLAKES.iloc[5,7],ETERNA.iloc[5,7],ETI.iloc[5,7],
                                                     FLOURMILL.iloc[5,7],LASACO.iloc[5,7],MANSARD.iloc[5,7],MBENEFIT.iloc[5,7],
                                                     MEYER.iloc[5,7],MORISON.iloc[5,7],MRS.iloc[5,7],NEM.iloc[5,7],
                                                     UACN.iloc[5,7],UCAP.iloc[5,7],UNIVINSURE.iloc[5,7],WAPIC.iloc[5,7],
                                                     VITAFOAM.iloc[5,7]],
                             
                            
                            'Change_in_Price':price6},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [56]:
february_2022

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,2.562370e+10,1.484132,-0.066667
FTNCOCOA,7.920000e+08,-0.948990,0.028571
LIVINGTRUST,6.200000e+09,0.520026,0.087719
ABCTRANS,5.304640e+08,1.215068,-0.111111
ACADEMY,1.070496e+09,0.269707,0.882979
...,...,...,...
UACN,2.953332e+10,1.721377,0.151685
UCAP,8.310000e+10,0.367591,0.259091
UNIVINSURE,3.200000e+09,2.923191,0.000000
WAPIC,1.151606e+10,1.954921,-0.040000


In [57]:
SL5=february_2022[(february_2022['Market_Cap']<=2000000000)&(february_2022['Book_to_market_ratio']>1)]

SM5=february_2022[(february_2022['Market_Cap']>2000000000) & (february_2022['Market_Cap']<10000000000) & (february_2022['Book_to_market_ratio']>1)]

SH5=february_2022[(february_2022['Market_Cap']>=10000000000)&(february_2022['Book_to_market_ratio']>1)]

BL5=february_2022[(february_2022['Market_Cap']<=2000000000) & (february_2022['Book_to_market_ratio']<1)]

BH5=february_2022[(february_2022['Market_Cap']>=10000000000)&(february_2022['Book_to_market_ratio']<1)]

BM5=february_2022[(february_2022['Market_Cap']>2000000000) & (february_2022['Market_Cap']<10000000000) & (february_2022['Book_to_market_ratio']<1)]


In [58]:
SLPRICE5=SL5['Change_in_Price'].mean()

SMPRICE5=SM5['Change_in_Price'].mean()

SHPRICE5=SH5['Change_in_Price'].mean()

BLPRICE5=BL5['Change_in_Price'].mean()

BMPRICE5=BM5['Change_in_Price'].mean()

BHPRICE5=BH5['Change_in_Price'].mean()



In [59]:
HML5=(1/3*(SLPRICE5+SMPRICE5+SHPRICE5)) - (1/3*(BLPRICE5 + BMPRICE5 + BHPRICE5))

SMB5=(1/2*(SHPRICE5 + BHPRICE5)) - (1/2*(SLPRICE5 +BLPRICE5))

HML5,SMB5

(-0.1449124397219538, -0.3741748948836933)

In [60]:
price7=new_df.iloc[13,:].values.flatten().tolist()

march_2022=pd.DataFrame({'Market_Cap':[AIICO.iloc[6,26],FTNCOCOA.iloc[6,26],LIVINGTRUST.iloc[6,26],ABCTRANS.iloc[6,26],
                                           ACADEMY.iloc[6,26],AFRINSURE.iloc[6,26],AFROMEDIA.iloc[6,6],AIRTELAFRI.iloc[6,26],
                                           ALEX.iloc[6,26],BERGER.iloc[6,26],BETAGLAS.iloc[6,26],BUACEMENT.iloc[6,26],
                                           BUAFOODS.iloc[6,26],CADBURY.iloc[6,26],CAP.iloc[6,26],CAVERTON.iloc[6,26],
                                           CHAMPION.iloc[6,26],CHAMS.iloc[6,26],CHELLARAM.iloc[6,26],CHIPLC.iloc[6,26],
                                           CILEASING.iloc[6,26],ETRANZACT.iloc[6,26],CONOIL.iloc[6,26],TRANSCORP.iloc[6,26],
                                           UNILEVER.iloc[6,26],UPDCREIT.iloc[6,26],UPL.iloc[6,26],VERITASKAP.iloc[6,26],
                                           ZENITHBANK.iloc[6,26],WAPCO.iloc[6,26],UPDC.iloc[6,26],UBN.iloc[6,26],
                                           PZ.iloc[6,26],LIVESTOCK.iloc[6,26],JAIZBANK.iloc[6,26],FCMB.iloc[6,26],
                                           FIDELITYBK.iloc[6,26],UBA.iloc[6,26],TOTAL.iloc[6,26],STANBIC.iloc[6,26],
                                           SOVRENINS.iloc[6,26],RTBRISCOE.iloc[6,26],SCOA.iloc[6,26],REDSTAREX.iloc[6,6],
                                           REGALINS.iloc[6,6],PRESCO.iloc[6,6],PHARMDEKO.iloc[6,6],OANDO.iloc[6,6],
                                           OKOMUOIL.iloc[6,6],NNFM.iloc[6,6],NGXGROUP.iloc[6,6],NESTLE.iloc[6,6],
                                           NEIMETH.iloc[6,6],NB.iloc[6,6],MULTIVERSE.iloc[6,6],NAHCO.iloc[6,6],NASCON.iloc[6,6],
                                           MTNN.iloc[6,6],MAYBAKER.iloc[6,6],MCNICHOLS.iloc[6,6],LEARNAFRCA.iloc[6,6],
                                           LINKASSURE.iloc[6,6],INTBREW.iloc[6,6],GUINNESS.iloc[6,6],HONYFLOUR.iloc[6,6],
                                           GLAXOSMITH.iloc[6,6],FBNH.iloc[6,6],AFRIPRUD.iloc[6,6],COURTVILLE.iloc[6,6],
                                           CUSTODIAN.iloc[6,6],CWG.iloc[6,6],DANGCEM.iloc[6,6],ELLAHLAKES.iloc[6,6],
                                           ETERNA.iloc[6,6],ETI.iloc[6,6],FLOURMILL.iloc[6,6],LASACO.iloc[6,6],
                                           MANSARD.iloc[6,6],MBENEFIT.iloc[6,6],MEYER.iloc[6,6],MORISON.iloc[6,6],MRS.iloc[6,6],
                                           NEM.iloc[6,6],UACN.iloc[6,6],UCAP.iloc[6,6],UNIVINSURE.iloc[6,6],WAPIC.iloc[6,6],
                                           VITAFOAM.iloc[6,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[6,27],FTNCOCOA.iloc[6,27],LIVINGTRUST.iloc[6,27],
                                                     ABCTRANS.iloc[6,27],ACADEMY.iloc[6,27],AFRINSURE.iloc[6,27],
                                                     AFROMEDIA.iloc[6,7],AIRTELAFRI.iloc[6,27],ALEX.iloc[6,27],BERGER.iloc[6,27],
                                                     BETAGLAS.iloc[6,27],BUACEMENT.iloc[6,27],BUAFOODS.iloc[6,27],
                                                     CADBURY.iloc[6,27],CAP.iloc[6,27],CAVERTON.iloc[6,27],CHAMPION.iloc[6,27],
                                                     CHAMS.iloc[6,27],CHELLARAM.iloc[6,27],CHIPLC.iloc[6,27],CILEASING.iloc[6,27],
                                                     ETRANZACT.iloc[6,27],CONOIL.iloc[6,27],TRANSCORP.iloc[6,27],
                                                     UNILEVER.iloc[6,27],UPDCREIT.iloc[6,27],UPL.iloc[6,27],VERITASKAP.iloc[6,27],
                                                     ZENITHBANK.iloc[6,27],WAPCO.iloc[6,27],UPDC.iloc[6,27],UBN.iloc[6,27],
                                                     PZ.iloc[6,27],LIVESTOCK.iloc[6,27],JAIZBANK.iloc[6,27],FCMB.iloc[6,27],
                                                     FIDELITYBK.iloc[6,27],UBA.iloc[6,27],TOTAL.iloc[6,27],STANBIC.iloc[6,27],
                                                     SOVRENINS.iloc[6,27],RTBRISCOE.iloc[6,27],SCOA.iloc[6,27],
                                                     REDSTAREX.iloc[6,7],
                                                     REGALINS.iloc[6,7],PRESCO.iloc[6,7],PHARMDEKO.iloc[6,7],OANDO.iloc[6,7],
                                                     OKOMUOIL.iloc[6,7],NNFM.iloc[6,7],NGXGROUP.iloc[6,7],NESTLE.iloc[6,7],
                                                     NEIMETH.iloc[6,7],NB.iloc[6,7],MULTIVERSE.iloc[6,7],NAHCO.iloc[6,7],
                                                     NASCON.iloc[6,7],MTNN.iloc[6,7],MAYBAKER.iloc[6,7],MCNICHOLS.iloc[6,7],
                                                     LEARNAFRCA.iloc[6,7],LINKASSURE.iloc[6,7],INTBREW.iloc[6,7],
                                                     GUINNESS.iloc[6,7],HONYFLOUR.iloc[6,7],GLAXOSMITH.iloc[6,7],FBNH.iloc[6,7],
                                                     AFRIPRUD.iloc[6,7],COURTVILLE.iloc[6,7],CUSTODIAN.iloc[6,7],CWG.iloc[6,7],
                                                     DANGCEM.iloc[6,7],ELLAHLAKES.iloc[6,7],ETERNA.iloc[6,7],ETI.iloc[6,7],
                                                     FLOURMILL.iloc[6,7],LASACO.iloc[6,7],MANSARD.iloc[6,7],MBENEFIT.iloc[6,7],
                                                     MEYER.iloc[6,7],MORISON.iloc[6,7],MRS.iloc[6,7],NEM.iloc[6,7],
                                                     UACN.iloc[6,7],UCAP.iloc[6,7],UNIVINSURE.iloc[6,7],WAPIC.iloc[6,7],
                                                     VITAFOAM.iloc[6,7]],
                             
                            
                            'Change_in_Price':price7},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [61]:
march_2022

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,2.452554e+10,1.759044,-0.042857
FTNCOCOA,7.040000e+08,-2.911591,-0.111111
LIVINGTRUST,6.200000e+09,0.520026,0.000000
ABCTRANS,4.973100e+08,1.197945,-0.062500
ACADEMY,1.070496e+09,0.173378,0.000000
...,...,...,...
UACN,3.443154e+10,1.495083,0.165854
UCAP,7.530000e+10,0.316722,-0.093863
UNIVINSURE,3.200000e+09,3.068106,0.000000
WAPIC,1.055639e+10,2.132642,-0.083333


In [62]:
SL6=march_2022[(march_2022['Market_Cap']<=2000000000)&(march_2022['Book_to_market_ratio']>1)]

SM6=march_2022[(march_2022['Market_Cap']>2000000000) & (march_2022['Market_Cap']<10000000000) & (march_2022['Book_to_market_ratio']>1)]

SH6=march_2022[(march_2022['Market_Cap']>=10000000000)&(march_2022['Book_to_market_ratio']>1)]

BL6=march_2022[(march_2022['Market_Cap']<=2000000000) & (march_2022['Book_to_market_ratio']<1)]

BH6=march_2022[(march_2022['Market_Cap']>=10000000000)&(march_2022['Book_to_market_ratio']<1)]

BM6=march_2022[(march_2022['Market_Cap']>2000000000) & (march_2022['Market_Cap']<10000000000) & (march_2022['Book_to_market_ratio']<1)]


In [63]:
SLPRICE6=SL6['Change_in_Price'].mean()

SMPRICE6=SM6['Change_in_Price'].mean()

SHPRICE6=SH6['Change_in_Price'].mean()

BLPRICE6=BL6['Change_in_Price'].mean()

BMPRICE6=BM6['Change_in_Price'].mean()

BHPRICE6=BH6['Change_in_Price'].mean()



In [64]:
HML6=(1/3*(SLPRICE6+SMPRICE6+SHPRICE6)) - (1/3*(BLPRICE6 + BMPRICE6 + BHPRICE6))

SMB6=(1/2*(SHPRICE6 + BHPRICE6)) - (1/2*(SLPRICE6 +BLPRICE6))

HML6,SMB6

(-0.007865416937987806, 0.06163360631333839)

In [65]:
price8=new_df.iloc[14,:].values.flatten().tolist()

april_2022=pd.DataFrame({'Market_Cap':[AIICO.iloc[7,26],FTNCOCOA.iloc[7,26],LIVINGTRUST.iloc[7,26],ABCTRANS.iloc[7,26],
                                           ACADEMY.iloc[7,26],AFRINSURE.iloc[7,26],AFROMEDIA.iloc[7,6],AIRTELAFRI.iloc[7,26],
                                           ALEX.iloc[7,26],BERGER.iloc[7,26],BETAGLAS.iloc[7,26],BUACEMENT.iloc[7,26],
                                           BUAFOODS.iloc[7,26],CADBURY.iloc[7,26],CAP.iloc[7,26],CAVERTON.iloc[7,26],
                                           CHAMPION.iloc[7,26],CHAMS.iloc[7,26],CHELLARAM.iloc[7,26],CHIPLC.iloc[7,26],
                                           CILEASING.iloc[7,26],ETRANZACT.iloc[7,26],CONOIL.iloc[7,26],TRANSCORP.iloc[7,26],
                                           UNILEVER.iloc[7,26],UPDCREIT.iloc[7,26],UPL.iloc[7,26],VERITASKAP.iloc[7,26],
                                           ZENITHBANK.iloc[7,26],WAPCO.iloc[7,26],UPDC.iloc[7,26],UBN.iloc[7,26],
                                           PZ.iloc[7,26],LIVESTOCK.iloc[7,26],JAIZBANK.iloc[7,26],FCMB.iloc[7,26],
                                           FIDELITYBK.iloc[7,26],UBA.iloc[7,26],TOTAL.iloc[7,26],STANBIC.iloc[7,26],
                                           SOVRENINS.iloc[7,26],RTBRISCOE.iloc[7,26],SCOA.iloc[7,26],REDSTAREX.iloc[7,6],
                                           REGALINS.iloc[7,6],PRESCO.iloc[7,6],PHARMDEKO.iloc[7,6],OANDO.iloc[7,6],
                                           OKOMUOIL.iloc[7,6],NNFM.iloc[7,6],NGXGROUP.iloc[7,6],NESTLE.iloc[7,6],
                                           NEIMETH.iloc[7,6],NB.iloc[7,6],MULTIVERSE.iloc[7,6],NAHCO.iloc[7,6],NASCON.iloc[7,6],
                                           MTNN.iloc[7,6],MAYBAKER.iloc[7,6],MCNICHOLS.iloc[7,6],LEARNAFRCA.iloc[7,6],
                                           LINKASSURE.iloc[7,6],INTBREW.iloc[7,6],GUINNESS.iloc[7,6],HONYFLOUR.iloc[7,6],
                                           GLAXOSMITH.iloc[7,6],FBNH.iloc[7,6],AFRIPRUD.iloc[7,6],COURTVILLE.iloc[7,6],
                                           CUSTODIAN.iloc[7,6],CWG.iloc[7,6],DANGCEM.iloc[7,6],ELLAHLAKES.iloc[7,6],
                                           ETERNA.iloc[7,6],ETI.iloc[7,6],FLOURMILL.iloc[7,6],LASACO.iloc[7,6],
                                       MANSARD.iloc[7,6],MBENEFIT.iloc[7,6],MEYER.iloc[7,6],MORISON.iloc[7,6],MRS.iloc[7,6],
                                           NEM.iloc[7,6],UACN.iloc[7,6],UCAP.iloc[7,6],UNIVINSURE.iloc[7,6],WAPIC.iloc[7,6],
                                           VITAFOAM.iloc[7,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[7,27],FTNCOCOA.iloc[7,27],LIVINGTRUST.iloc[7,27],
                                                     ABCTRANS.iloc[7,27],ACADEMY.iloc[7,27],AFRINSURE.iloc[7,27],
                                                     AFROMEDIA.iloc[7,7],AIRTELAFRI.iloc[7,27],ALEX.iloc[7,27],BERGER.iloc[7,27],
                                                     BETAGLAS.iloc[7,27],BUACEMENT.iloc[7,27],BUAFOODS.iloc[7,27],
                                                     CADBURY.iloc[7,27],CAP.iloc[7,27],CAVERTON.iloc[7,27],CHAMPION.iloc[7,27],
                                                     CHAMS.iloc[7,27],CHELLARAM.iloc[7,27],CHIPLC.iloc[7,27],CILEASING.iloc[7,27],
                                                     ETRANZACT.iloc[7,27],CONOIL.iloc[7,27],TRANSCORP.iloc[7,27],
                                                     UNILEVER.iloc[7,27],UPDCREIT.iloc[7,27],UPL.iloc[7,27],VERITASKAP.iloc[7,27],
                                                     ZENITHBANK.iloc[7,27],WAPCO.iloc[7,27],UPDC.iloc[7,27],UBN.iloc[7,27],
                                                     PZ.iloc[7,27],LIVESTOCK.iloc[7,27],JAIZBANK.iloc[7,27],FCMB.iloc[7,27],
                                                     FIDELITYBK.iloc[7,27],UBA.iloc[7,27],TOTAL.iloc[7,27],STANBIC.iloc[7,27],
                                                     SOVRENINS.iloc[7,27],RTBRISCOE.iloc[7,27],SCOA.iloc[7,27],
                                                     REDSTAREX.iloc[7,7],
                                                     REGALINS.iloc[7,7],PRESCO.iloc[7,7],PHARMDEKO.iloc[7,7],OANDO.iloc[7,7],
                                                     OKOMUOIL.iloc[7,7],NNFM.iloc[7,7],NGXGROUP.iloc[7,7],NESTLE.iloc[7,7],
                                                     NEIMETH.iloc[7,7],NB.iloc[7,7],MULTIVERSE.iloc[7,7],NAHCO.iloc[7,7],
                                                     NASCON.iloc[7,7],MTNN.iloc[7,7],MAYBAKER.iloc[7,7],MCNICHOLS.iloc[7,7],
                                                     LEARNAFRCA.iloc[7,7],LINKASSURE.iloc[7,7],INTBREW.iloc[7,7],
                                                     GUINNESS.iloc[7,7],HONYFLOUR.iloc[7,7],GLAXOSMITH.iloc[7,7],FBNH.iloc[7,7],
                                                     AFRIPRUD.iloc[7,7],COURTVILLE.iloc[7,7],CUSTODIAN.iloc[7,7],CWG.iloc[7,7],
                                                     DANGCEM.iloc[7,7],ELLAHLAKES.iloc[7,7],ETERNA.iloc[7,7],ETI.iloc[7,7],
                                                     FLOURMILL.iloc[7,7],LASACO.iloc[7,7],MANSARD.iloc[7,7],MBENEFIT.iloc[7,7],
                                                     MEYER.iloc[7,7],MORISON.iloc[7,7],MRS.iloc[7,7],NEM.iloc[7,7],
                                                     UACN.iloc[7,7],UCAP.iloc[7,7],UNIVINSURE.iloc[7,7],WAPIC.iloc[7,7],
                                                     VITAFOAM.iloc[7,7]],
                             
                            
                            'Change_in_Price':price8},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [66]:
april_2022

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,2.891817e+10,1.491847,0.179104
FTNCOCOA,8.140000e+08,-2.518133,0.156250
LIVINGTRUST,6.200000e+09,0.520026,0.000000
ABCTRANS,5.138870e+08,1.159302,0.033333
ACADEMY,8.890560e+08,0.208761,-0.169492
...,...,...,...
UACN,3.457560e+10,1.488853,0.004184
UCAP,8.070000e+10,0.295529,0.071713
UNIVINSURE,3.200000e+09,3.068106,0.000000
WAPIC,1.055639e+10,2.132642,0.000000


In [67]:
SL7=april_2022[(april_2022['Market_Cap']<=2000000000)&(april_2022['Book_to_market_ratio']>1)]

SM7=april_2022[(april_2022['Market_Cap']>2000000000) & (april_2022['Market_Cap']<10000000000) & (april_2022['Book_to_market_ratio']>1)]

SH7=april_2022[(april_2022['Market_Cap']>=10000000000)&(april_2022['Book_to_market_ratio']>1)]

BL7=april_2022[(april_2022['Market_Cap']<=2000000000) & (april_2022['Book_to_market_ratio']<1)]

BH7=april_2022[(april_2022['Market_Cap']>=10000000000)&(april_2022['Book_to_market_ratio']<1)]

BM7=april_2022[(april_2022['Market_Cap']>2000000000) & (april_2022['Market_Cap']<10000000000) & (april_2022['Book_to_market_ratio']<1)]


In [68]:
SLPRICE7=SL7['Change_in_Price'].mean()

SMPRICE7=SM7['Change_in_Price'].mean()

SHPRICE7=SH7['Change_in_Price'].mean()

BLPRICE7=BL7['Change_in_Price'].mean()

BMPRICE7=BM7['Change_in_Price'].mean()

BHPRICE7=BH7['Change_in_Price'].mean()



In [69]:
HML7=(1/3*(SLPRICE7+SMPRICE7+SHPRICE7)) - (1/3*(BLPRICE7 + BMPRICE7 + BHPRICE7))

SMB7=(1/2*(SHPRICE7 + BHPRICE7)) - (1/2*(SLPRICE7 +BLPRICE7))

HML7,SMB7

(-0.1531469926168128, 0.0912599374741606)

In [70]:
price9=new_df.iloc[15,:].values.flatten().tolist()#prices from new_df,  monthly stock variable
may_2022=pd.DataFrame({'Market_Cap':[AIICO.iloc[8,26],FTNCOCOA.iloc[8,26],LIVINGTRUST.iloc[8,26],ABCTRANS.iloc[8,26],
                                           ACADEMY.iloc[8,26],AFRINSURE.iloc[8,26],AFROMEDIA.iloc[8,6],AIRTELAFRI.iloc[8,26],
                                           ALEX.iloc[8,26],BERGER.iloc[8,26],BETAGLAS.iloc[8,26],BUACEMENT.iloc[8,26],
                                           BUAFOODS.iloc[8,26],CADBURY.iloc[8,26],CAP.iloc[8,26],CAVERTON.iloc[8,26],
                                           CHAMPION.iloc[8,26],CHAMS.iloc[8,26],CHELLARAM.iloc[8,26],CHIPLC.iloc[8,26],
                                           CILEASING.iloc[8,26],ETRANZACT.iloc[8,26],CONOIL.iloc[8,26],TRANSCORP.iloc[8,26],
                                           UNILEVER.iloc[8,26],UPDCREIT.iloc[8,26],UPL.iloc[8,26],VERITASKAP.iloc[8,26],
                                           ZENITHBANK.iloc[8,26],WAPCO.iloc[8,26],UPDC.iloc[8,26],UBN.iloc[8,26],
                                           PZ.iloc[8,26],LIVESTOCK.iloc[8,26],JAIZBANK.iloc[8,26],FCMB.iloc[8,26],
                                           FIDELITYBK.iloc[8,26],UBA.iloc[8,26],TOTAL.iloc[8,26],STANBIC.iloc[8,26],
                                           SOVRENINS.iloc[8,26],RTBRISCOE.iloc[8,26],SCOA.iloc[8,26],REDSTAREX.iloc[8,6],
                                           REGALINS.iloc[8,6],PRESCO.iloc[8,6],PHARMDEKO.iloc[8,6],OANDO.iloc[8,6],
                                           OKOMUOIL.iloc[8,6],NNFM.iloc[8,6],NGXGROUP.iloc[8,6],NESTLE.iloc[8,6],
                                           NEIMETH.iloc[8,6],NB.iloc[8,6],MULTIVERSE.iloc[8,6],NAHCO.iloc[8,6],NASCON.iloc[8,6],
                                           MTNN.iloc[8,6],MAYBAKER.iloc[8,6],MCNICHOLS.iloc[8,6],LEARNAFRCA.iloc[8,6],
                                           LINKASSURE.iloc[8,6],INTBREW.iloc[8,6],GUINNESS.iloc[8,6],HONYFLOUR.iloc[8,6],
                                           GLAXOSMITH.iloc[8,6],FBNH.iloc[8,6],AFRIPRUD.iloc[8,6],COURTVILLE.iloc[8,6],
                                           CUSTODIAN.iloc[8,6],CWG.iloc[8,6],DANGCEM.iloc[8,6],ELLAHLAKES.iloc[8,6],
                                           ETERNA.iloc[8,6],ETI.iloc[8,6],FLOURMILL.iloc[8,6],LASACO.iloc[8,6],
                                           MANSARD.iloc[8,6],MBENEFIT.iloc[8,6],MEYER.iloc[8,6],MORISON.iloc[8,6],MRS.iloc[8,6],
                                           NEM.iloc[8,6],UACN.iloc[8,6],UCAP.iloc[8,6],UNIVINSURE.iloc[8,6],WAPIC.iloc[8,6],
                                           VITAFOAM.iloc[8,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[8,27],FTNCOCOA.iloc[8,27],LIVINGTRUST.iloc[8,27],
                                                     ABCTRANS.iloc[8,27],ACADEMY.iloc[8,27],AFRINSURE.iloc[8,27],
                                                     AFROMEDIA.iloc[8,7],AIRTELAFRI.iloc[8,27],ALEX.iloc[8,27],BERGER.iloc[8,27],
                                                     BETAGLAS.iloc[8,27],BUACEMENT.iloc[8,27],BUAFOODS.iloc[8,27],
                                                     CADBURY.iloc[8,27],CAP.iloc[8,27],CAVERTON.iloc[8,27],CHAMPION.iloc[8,27],
                                                     CHAMS.iloc[8,27],CHELLARAM.iloc[8,27],CHIPLC.iloc[8,27],CILEASING.iloc[8,27],
                                                     ETRANZACT.iloc[8,27],CONOIL.iloc[8,27],TRANSCORP.iloc[8,27],
                                                     UNILEVER.iloc[8,27],UPDCREIT.iloc[8,27],UPL.iloc[8,27],VERITASKAP.iloc[8,27],
                                                     ZENITHBANK.iloc[8,27],WAPCO.iloc[8,27],UPDC.iloc[8,27],UBN.iloc[8,27],
                                                     PZ.iloc[8,27],LIVESTOCK.iloc[8,27],JAIZBANK.iloc[8,27],FCMB.iloc[8,27],
                                                     FIDELITYBK.iloc[8,27],UBA.iloc[8,27],TOTAL.iloc[8,27],STANBIC.iloc[8,27],
                                                     SOVRENINS.iloc[8,27],RTBRISCOE.iloc[8,27],SCOA.iloc[8,27],
                                                     REDSTAREX.iloc[8,7],
                                                     REGALINS.iloc[8,7],PRESCO.iloc[8,7],PHARMDEKO.iloc[8,7],OANDO.iloc[8,7],
                                                     OKOMUOIL.iloc[8,7],NNFM.iloc[8,7],NGXGROUP.iloc[8,7],NESTLE.iloc[8,7],
                                                     NEIMETH.iloc[8,7],NB.iloc[8,7],MULTIVERSE.iloc[8,7],NAHCO.iloc[8,7],
                                                     NASCON.iloc[8,7],MTNN.iloc[8,7],MAYBAKER.iloc[8,7],MCNICHOLS.iloc[8,7],
                                                     LEARNAFRCA.iloc[8,7],LINKASSURE.iloc[8,7],INTBREW.iloc[8,7],
                                                     GUINNESS.iloc[8,7],HONYFLOUR.iloc[8,7],GLAXOSMITH.iloc[8,7],FBNH.iloc[8,7],
                                                     AFRIPRUD.iloc[8,7],COURTVILLE.iloc[8,7],CUSTODIAN.iloc[8,7],CWG.iloc[8,7],
                                                     DANGCEM.iloc[8,7],ELLAHLAKES.iloc[8,7],ETERNA.iloc[8,7],ETI.iloc[8,7],
                                                     FLOURMILL.iloc[8,7],LASACO.iloc[8,7],MANSARD.iloc[8,7],MBENEFIT.iloc[8,7],
                                                     MEYER.iloc[8,7],MORISON.iloc[8,7],MRS.iloc[8,7],NEM.iloc[8,7],
                                                     UACN.iloc[8,7],UCAP.iloc[8,7],UNIVINSURE.iloc[8,7],WAPIC.iloc[8,7],
                                                     VITAFOAM.iloc[8,7]],
                             
                            
                            'Change_in_Price':price9},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [71]:
may_2022

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,2.525764e+10,1.708057,-0.126582
FTNCOCOA,7.040000e+08,-2.911591,-0.135135
LIVINGTRUST,5.700000e+09,0.565642,-0.080645
ABCTRANS,5.470410e+08,1.089041,0.064516
ACADEMY,7.983360e+08,0.232484,-0.102041
...,...,...,...
UACN,3.313495e+10,1.553586,-0.041667
UCAP,7.800000e+10,0.305759,-0.033457
UNIVINSURE,3.200000e+09,3.068106,0.000000
WAPIC,1.055639e+10,2.132642,0.000000


In [72]:
SL8=may_2022[(may_2022['Market_Cap']<=2000000000)&(may_2022['Book_to_market_ratio']>1)]

SM8=may_2022[(may_2022['Market_Cap']>2000000000) & (may_2022['Market_Cap']<10000000000) & (may_2022['Book_to_market_ratio']>1)]

SH8=may_2022[(may_2022['Market_Cap']>=10000000000)&(may_2022['Book_to_market_ratio']>1)]

BL8=may_2022[(may_2022['Market_Cap']<=2000000000) & (may_2022['Book_to_market_ratio']<1)]

BH8=may_2022[(may_2022['Market_Cap']>=10000000000)&(may_2022['Book_to_market_ratio']<1)]

BM8=may_2022[(may_2022['Market_Cap']>2000000000) & (may_2022['Market_Cap']<10000000000) & (may_2022['Book_to_market_ratio']<1)]


In [73]:
SLPRICE8=SL8['Change_in_Price'].mean()

SMPRICE8=SM8['Change_in_Price'].mean()

SHPRICE8=SH8['Change_in_Price'].mean()

BLPRICE8=BL8['Change_in_Price'].mean()

BMPRICE8=BM8['Change_in_Price'].mean()

BHPRICE8=BH8['Change_in_Price'].mean()



In [74]:
HML8=(1/3*(SLPRICE8+SMPRICE8+SHPRICE8)) - (1/3*(BLPRICE8 + BMPRICE8 + BHPRICE8))

SMB8=(1/2*(SHPRICE8 + BHPRICE8)) - (1/2*(SLPRICE8 +BLPRICE8))

HML8,SMB8

(0.061007304911009724, -0.01642784331194861)

In [75]:
price10=new_df.iloc[16,:].values.flatten().tolist()#prices from new_df,  monthly stock variable
june_2022=pd.DataFrame({'Market_Cap':[AIICO.iloc[9,26],FTNCOCOA.iloc[9,26],LIVINGTRUST.iloc[9,26],ABCTRANS.iloc[9,26],
                                           ACADEMY.iloc[9,26],AFRINSURE.iloc[9,26],AFROMEDIA.iloc[9,6],AIRTELAFRI.iloc[9,26],
                                           ALEX.iloc[9,26],BERGER.iloc[9,26],BETAGLAS.iloc[9,26],BUACEMENT.iloc[9,26],
                                           BUAFOODS.iloc[9,26],CADBURY.iloc[9,26],CAP.iloc[9,26],CAVERTON.iloc[9,26],
                                           CHAMPION.iloc[9,26],CHAMS.iloc[9,26],CHELLARAM.iloc[9,26],CHIPLC.iloc[9,26],
                                           CILEASING.iloc[9,26],ETRANZACT.iloc[9,26],CONOIL.iloc[9,26],TRANSCORP.iloc[9,26],
                                           UNILEVER.iloc[9,26],UPDCREIT.iloc[9,26],UPL.iloc[9,26],VERITASKAP.iloc[9,26],
                                           ZENITHBANK.iloc[9,26],WAPCO.iloc[9,26],UPDC.iloc[9,26],UBN.iloc[9,26],
                                           PZ.iloc[9,26],LIVESTOCK.iloc[9,26],JAIZBANK.iloc[9,26],FCMB.iloc[9,26],
                                           FIDELITYBK.iloc[9,26],UBA.iloc[9,26],TOTAL.iloc[9,26],STANBIC.iloc[9,26],
                                           SOVRENINS.iloc[9,26],RTBRISCOE.iloc[9,26],SCOA.iloc[9,26],REDSTAREX.iloc[9,6],
                                           REGALINS.iloc[9,6],PRESCO.iloc[9,6],PHARMDEKO.iloc[9,6],OANDO.iloc[9,6],
                                           OKOMUOIL.iloc[9,6],NNFM.iloc[9,6],NGXGROUP.iloc[9,6],NESTLE.iloc[9,6],
                                           NEIMETH.iloc[9,6],NB.iloc[9,6],MULTIVERSE.iloc[9,6],NAHCO.iloc[9,6],NASCON.iloc[9,6],
                                           MTNN.iloc[9,6],MAYBAKER.iloc[9,6],MCNICHOLS.iloc[9,6],LEARNAFRCA.iloc[9,6],
                                           LINKASSURE.iloc[9,6],INTBREW.iloc[9,6],GUINNESS.iloc[9,6],HONYFLOUR.iloc[9,6],
                                           GLAXOSMITH.iloc[9,6],FBNH.iloc[9,6],AFRIPRUD.iloc[9,6],COURTVILLE.iloc[9,6],
                                           CUSTODIAN.iloc[9,6],CWG.iloc[9,6],DANGCEM.iloc[9,6],ELLAHLAKES.iloc[9,6],
                                           ETERNA.iloc[9,6],ETI.iloc[9,6],FLOURMILL.iloc[9,6],LASACO.iloc[9,6],
                                           MANSARD.iloc[9,6],MBENEFIT.iloc[9,6],MEYER.iloc[9,6],MORISON.iloc[9,6],MRS.iloc[9,6],
                                           NEM.iloc[9,6],UACN.iloc[9,6],UCAP.iloc[9,6],UNIVINSURE.iloc[9,6],WAPIC.iloc[9,6],
                                           VITAFOAM.iloc[9,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[9,27],FTNCOCOA.iloc[9,27],LIVINGTRUST.iloc[9,27],
                                                     ABCTRANS.iloc[9,27],ACADEMY.iloc[9,27],AFRINSURE.iloc[9,27],
                                                     AFROMEDIA.iloc[9,7],AIRTELAFRI.iloc[9,27],ALEX.iloc[9,27],BERGER.iloc[9,27],
                                                     BETAGLAS.iloc[9,27],BUACEMENT.iloc[9,27],BUAFOODS.iloc[9,27],
                                                     CADBURY.iloc[9,27],CAP.iloc[9,27],CAVERTON.iloc[9,27],CHAMPION.iloc[9,27],
                                                     CHAMS.iloc[9,27],CHELLARAM.iloc[9,27],CHIPLC.iloc[9,27],CILEASING.iloc[9,27],
                                                     ETRANZACT.iloc[9,27],CONOIL.iloc[9,27],TRANSCORP.iloc[9,27],
                                                     UNILEVER.iloc[9,27],UPDCREIT.iloc[9,27],UPL.iloc[9,27],VERITASKAP.iloc[9,27],
                                                     ZENITHBANK.iloc[9,27],WAPCO.iloc[9,27],UPDC.iloc[9,27],UBN.iloc[9,27],
                                                     PZ.iloc[9,27],LIVESTOCK.iloc[9,27],JAIZBANK.iloc[9,27],FCMB.iloc[9,27],
                                                     FIDELITYBK.iloc[9,27],UBA.iloc[9,27],TOTAL.iloc[9,27],STANBIC.iloc[9,27],
                                                     SOVRENINS.iloc[9,27],RTBRISCOE.iloc[9,27],SCOA.iloc[9,27],
                                                     REDSTAREX.iloc[9,7],
                                                     REGALINS.iloc[9,7],PRESCO.iloc[9,7],PHARMDEKO.iloc[9,7],OANDO.iloc[9,7],
                                                     OKOMUOIL.iloc[9,7],NNFM.iloc[9,7],NGXGROUP.iloc[9,7],NESTLE.iloc[9,7],
                                                     NEIMETH.iloc[9,7],NB.iloc[9,7],MULTIVERSE.iloc[9,7],NAHCO.iloc[9,7],
                                                     NASCON.iloc[9,7],MTNN.iloc[9,7],MAYBAKER.iloc[9,7],MCNICHOLS.iloc[9,7],
                                                     LEARNAFRCA.iloc[9,7],LINKASSURE.iloc[9,7],INTBREW.iloc[9,7],
                                                     GUINNESS.iloc[9,7],HONYFLOUR.iloc[9,7],GLAXOSMITH.iloc[9,7],FBNH.iloc[9,7],
                                                     AFRIPRUD.iloc[9,7],COURTVILLE.iloc[9,7],CUSTODIAN.iloc[9,7],CWG.iloc[9,7],
                                                     DANGCEM.iloc[9,7],ELLAHLAKES.iloc[9,7],ETERNA.iloc[9,7],ETI.iloc[9,7],
                                                     FLOURMILL.iloc[9,7],LASACO.iloc[9,7],MANSARD.iloc[9,7],MBENEFIT.iloc[9,7],
                                                     MEYER.iloc[9,7],MORISON.iloc[9,7],MRS.iloc[9,7],NEM.iloc[9,7],
                                                     UACN.iloc[9,7],UCAP.iloc[9,7],UNIVINSURE.iloc[9,7],WAPIC.iloc[9,7],
                                                     VITAFOAM.iloc[9,7]],
                             
                            
                            'Change_in_Price':price10},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [76]:
june_2022

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,2.342738e+10,1.830401,-0.072464
FTNCOCOA,7.260000e+08,-2.934160,0.031250
LIVINGTRUST,6.000000e+09,0.487360,0.052632
ABCTRANS,4.973100e+08,1.197945,-0.090909
ACADEMY,7.680960e+08,0.516342,-0.037879
...,...,...,...
UACN,3.198243e+10,1.609571,-0.034783
UCAP,7.650000e+10,0.311754,-0.019231
UNIVINSURE,3.200000e+09,3.068106,0.000000
WAPIC,9.836638e+09,2.103869,-0.068182


In [77]:
SL9=june_2022[(june_2022['Market_Cap']<=2000000000)&(june_2022['Book_to_market_ratio']>1)]

SM9=june_2022[(june_2022['Market_Cap']>2000000000) & (june_2022['Market_Cap']<10000000000) & (june_2022['Book_to_market_ratio']>1)]

SH9=june_2022[(june_2022['Market_Cap']>=10000000000)&(june_2022['Book_to_market_ratio']>1)]

BL9=june_2022[(june_2022['Market_Cap']<=2000000000) & (june_2022['Book_to_market_ratio']<1)]

BH9=june_2022[(june_2022['Market_Cap']>=10000000000)&(june_2022['Book_to_market_ratio']<1)]

BM9=june_2022[(june_2022['Market_Cap']>2000000000) & (june_2022['Market_Cap']<10000000000) & (june_2022['Book_to_market_ratio']<1)]


In [78]:
SLPRICE9=SL9['Change_in_Price'].mean()

SMPRICE9=SM9['Change_in_Price'].mean()

SHPRICE9=SH9['Change_in_Price'].mean()

BLPRICE9=BL9['Change_in_Price'].mean()

BMPRICE9=BM9['Change_in_Price'].mean()

BHPRICE9=BH9['Change_in_Price'].mean()



In [79]:
HML9=(1/3*(SLPRICE9+SMPRICE9+SHPRICE9)) - (1/3*(BLPRICE9 + BMPRICE9 + BHPRICE9))

SMB9=(1/2*(SHPRICE9 + BHPRICE9)) - (1/2*(SLPRICE9 +BLPRICE9))

HML9,SMB9

(0.005400704661600624, -0.020102698657305185)

In [80]:
price11=new_df.iloc[17,:].values.flatten().tolist()#prices from new_df,  monthly stock variable
july_2022=pd.DataFrame({'Market_Cap':[AIICO.iloc[10,26],FTNCOCOA.iloc[10,26],LIVINGTRUST.iloc[10,26],ABCTRANS.iloc[10,26],
                                           ACADEMY.iloc[10,26],AFRINSURE.iloc[10,26],AFROMEDIA.iloc[10,6],AIRTELAFRI.iloc[10,26],
                                           ALEX.iloc[10,26],BERGER.iloc[10,26],BETAGLAS.iloc[10,26],BUACEMENT.iloc[10,26],
                                           BUAFOODS.iloc[10,26],CADBURY.iloc[10,26],CAP.iloc[10,26],CAVERTON.iloc[10,26],
                                           CHAMPION.iloc[10,26],CHAMS.iloc[10,26],CHELLARAM.iloc[10,26],CHIPLC.iloc[10,26],
                                           CILEASING.iloc[10,26],ETRANZACT.iloc[10,26],CONOIL.iloc[10,26],TRANSCORP.iloc[10,26],
                                           UNILEVER.iloc[10,26],UPDCREIT.iloc[10,26],UPL.iloc[10,26],VERITASKAP.iloc[10,26],
                                           ZENITHBANK.iloc[10,26],WAPCO.iloc[10,26],UPDC.iloc[10,26],UBN.iloc[10,26],
                                           PZ.iloc[10,26],LIVESTOCK.iloc[10,26],JAIZBANK.iloc[10,26],FCMB.iloc[10,26],
                                           FIDELITYBK.iloc[10,26],UBA.iloc[10,26],TOTAL.iloc[10,26],STANBIC.iloc[10,26],
                                           SOVRENINS.iloc[10,26],RTBRISCOE.iloc[10,26],SCOA.iloc[10,26],REDSTAREX.iloc[10,6],
                                           REGALINS.iloc[10,6],PRESCO.iloc[10,6],PHARMDEKO.iloc[10,6],OANDO.iloc[10,6],
                                           OKOMUOIL.iloc[10,6],NNFM.iloc[10,6],NGXGROUP.iloc[10,6],NESTLE.iloc[10,6],
                                           NEIMETH.iloc[10,6],NB.iloc[10,6],MULTIVERSE.iloc[10,6],NAHCO.iloc[10,6],NASCON.iloc[10,6],
                                           MTNN.iloc[10,6],MAYBAKER.iloc[10,6],MCNICHOLS.iloc[10,6],LEARNAFRCA.iloc[10,6],
                                           LINKASSURE.iloc[10,6],INTBREW.iloc[10,6],GUINNESS.iloc[10,6],HONYFLOUR.iloc[10,6],
                                           GLAXOSMITH.iloc[10,6],FBNH.iloc[10,6],AFRIPRUD.iloc[10,6],COURTVILLE.iloc[10,6],
                                           CUSTODIAN.iloc[10,6],CWG.iloc[10,6],DANGCEM.iloc[10,6],ELLAHLAKES.iloc[10,6],
                                           ETERNA.iloc[10,6],ETI.iloc[10,6],FLOURMILL.iloc[10,6],LASACO.iloc[10,6],
                                           MANSARD.iloc[10,6],MBENEFIT.iloc[10,6],MEYER.iloc[10,6],MORISON.iloc[10,6],MRS.iloc[10,6],
                                           NEM.iloc[10,6],UACN.iloc[10,6],UCAP.iloc[10,6],UNIVINSURE.iloc[10,6],WAPIC.iloc[10,6],
                                           VITAFOAM.iloc[10,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[10,27],FTNCOCOA.iloc[10,27],LIVINGTRUST.iloc[10,27],
                                                     ABCTRANS.iloc[10,27],ACADEMY.iloc[10,27],AFRINSURE.iloc[10,27],
                                                     AFROMEDIA.iloc[10,7],AIRTELAFRI.iloc[10,27],ALEX.iloc[10,27],BERGER.iloc[10,27],
                                                     BETAGLAS.iloc[10,27],BUACEMENT.iloc[10,27],BUAFOODS.iloc[10,27],
                                                     CADBURY.iloc[10,27],CAP.iloc[10,27],CAVERTON.iloc[10,27],CHAMPION.iloc[10,27],
                                                     CHAMS.iloc[10,27],CHELLARAM.iloc[10,27],CHIPLC.iloc[10,27],CILEASING.iloc[10,27],
                                                     ETRANZACT.iloc[10,27],CONOIL.iloc[10,27],TRANSCORP.iloc[10,27],
                                                     UNILEVER.iloc[10,27],UPDCREIT.iloc[10,27],UPL.iloc[10,27],VERITASKAP.iloc[10,27],
                                                     ZENITHBANK.iloc[10,27],WAPCO.iloc[10,27],UPDC.iloc[10,27],UBN.iloc[10,27],
                                                     PZ.iloc[10,27],LIVESTOCK.iloc[10,27],JAIZBANK.iloc[10,27],FCMB.iloc[10,27],
                                                     FIDELITYBK.iloc[10,27],UBA.iloc[10,27],TOTAL.iloc[10,27],STANBIC.iloc[10,27],
                                                     SOVRENINS.iloc[10,27],RTBRISCOE.iloc[10,27],SCOA.iloc[10,27],
                                                     REDSTAREX.iloc[10,7],
                                                     REGALINS.iloc[10,7],PRESCO.iloc[10,7],PHARMDEKO.iloc[10,7],OANDO.iloc[10,7],
                                                     OKOMUOIL.iloc[10,7],NNFM.iloc[10,7],NGXGROUP.iloc[10,7],NESTLE.iloc[10,7],
                                                     NEIMETH.iloc[10,7],NB.iloc[10,7],MULTIVERSE.iloc[10,7],NAHCO.iloc[10,7],
                                                     NASCON.iloc[10,7],MTNN.iloc[10,7],MAYBAKER.iloc[10,7],MCNICHOLS.iloc[10,7],
                                                     LEARNAFRCA.iloc[10,7],LINKASSURE.iloc[10,7],INTBREW.iloc[10,7],
                                                     GUINNESS.iloc[10,7],HONYFLOUR.iloc[10,7],GLAXOSMITH.iloc[10,7],FBNH.iloc[10,7],
                                                     AFRIPRUD.iloc[10,7],COURTVILLE.iloc[10,7],CUSTODIAN.iloc[10,7],CWG.iloc[10,7],
                                                     DANGCEM.iloc[10,7],ELLAHLAKES.iloc[10,7],ETERNA.iloc[10,7],ETI.iloc[10,7],
                                                     FLOURMILL.iloc[10,7],LASACO.iloc[10,7],MANSARD.iloc[10,7],MBENEFIT.iloc[10,7],
                                                     MEYER.iloc[10,7],MORISON.iloc[10,7],MRS.iloc[10,7],NEM.iloc[10,7],
                                                     UACN.iloc[10,7],UCAP.iloc[10,7],UNIVINSURE.iloc[10,7],WAPIC.iloc[10,7],
                                                     VITAFOAM.iloc[10,7]],
                             
                            
                            'Change_in_Price':price11},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [81]:
july_2022

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,2.086501e+10,2.055187,-0.109375
FTNCOCOA,6.600000e+08,-3.227576,-0.090909
LIVINGTRUST,6.000000e+09,0.487360,0.000000
ABCTRANS,4.973100e+08,1.197945,0.000000
ACADEMY,1.118880e+09,0.354462,0.456693
...,...,...,...
UACN,2.766048e+10,1.861067,0.009009
UCAP,6.660000e+10,0.358096,-0.129412
UNIVINSURE,3.200000e+09,3.068106,0.000000
WAPIC,9.116884e+09,2.269964,0.000000


In [82]:
SL10=july_2022[(july_2022['Market_Cap']<=2000000000)&(july_2022['Book_to_market_ratio']>1)]

SM10=july_2022[(july_2022['Market_Cap']>2000000000) & (july_2022['Market_Cap']<10000000000) & (july_2022['Book_to_market_ratio']>1)]

SH10=july_2022[(july_2022['Market_Cap']>=10000000000)&(july_2022['Book_to_market_ratio']>1)]

BL10=july_2022[(july_2022['Market_Cap']<=2000000000) & (july_2022['Book_to_market_ratio']<1)]

BH10=july_2022[(july_2022['Market_Cap']>=10000000000)&(july_2022['Book_to_market_ratio']<1)]

BM10=july_2022[(july_2022['Market_Cap']>2000000000) & (july_2022['Market_Cap']<10000000000) & (july_2022['Book_to_market_ratio']<1)]


In [83]:
SLPRICE10=SL10['Change_in_Price'].mean()

SMPRICE10=SM10['Change_in_Price'].mean()

SHPRICE10=SH10['Change_in_Price'].mean()

BLPRICE10=BL10['Change_in_Price'].mean()

BMPRICE10=BM10['Change_in_Price'].mean()

BHPRICE10=BH10['Change_in_Price'].mean()



In [84]:
HML10=(1/3*(SLPRICE10+SMPRICE10+SHPRICE10)) - (1/3*(BLPRICE10 + BMPRICE10 + BHPRICE10))

SMB10=(1/2*(SHPRICE10 + BHPRICE10)) - (1/2*(SLPRICE10 +BLPRICE10))

HML10,SMB10

(-0.04061457563000022, -0.07237649934806499)

In [85]:
price12=new_df.iloc[18,:].values.flatten().tolist()#prices from new_df,  monthly stock variable
august_2022=pd.DataFrame({'Market_Cap':[AIICO.iloc[11,26],FTNCOCOA.iloc[11,26],LIVINGTRUST.iloc[11,26],ABCTRANS.iloc[11,26],
                                           ACADEMY.iloc[11,26],AFRINSURE.iloc[11,26],AFROMEDIA.iloc[11,6],AIRTELAFRI.iloc[11,26],
                                           ALEX.iloc[11,26],BERGER.iloc[11,26],BETAGLAS.iloc[11,26],BUACEMENT.iloc[11,26],
                                           BUAFOODS.iloc[11,26],CADBURY.iloc[11,26],CAP.iloc[11,26],CAVERTON.iloc[11,26],
                                           CHAMPION.iloc[11,26],CHAMS.iloc[11,26],CHELLARAM.iloc[11,26],CHIPLC.iloc[11,26],
                                           CILEASING.iloc[11,26],ETRANZACT.iloc[11,26],CONOIL.iloc[11,26],TRANSCORP.iloc[11,26],
                                           UNILEVER.iloc[11,26],UPDCREIT.iloc[11,26],UPL.iloc[11,26],VERITASKAP.iloc[11,26],
                                           ZENITHBANK.iloc[11,26],WAPCO.iloc[11,26],UPDC.iloc[11,26],UBN.iloc[11,26],
                                           PZ.iloc[11,26],LIVESTOCK.iloc[11,26],JAIZBANK.iloc[11,26],FCMB.iloc[11,26],
                                           FIDELITYBK.iloc[11,26],UBA.iloc[11,26],TOTAL.iloc[11,26],STANBIC.iloc[11,26],
                                           SOVRENINS.iloc[11,26],RTBRISCOE.iloc[11,26],SCOA.iloc[11,26],REDSTAREX.iloc[11,6],
                                           REGALINS.iloc[11,6],PRESCO.iloc[11,6],PHARMDEKO.iloc[11,6],OANDO.iloc[11,6],
                                           OKOMUOIL.iloc[11,6],NNFM.iloc[11,6],NGXGROUP.iloc[11,6],NESTLE.iloc[11,6],
                                           NEIMETH.iloc[11,6],NB.iloc[11,6],MULTIVERSE.iloc[11,6],NAHCO.iloc[11,6],NASCON.iloc[11,6],
                                           MTNN.iloc[11,6],MAYBAKER.iloc[11,6],MCNICHOLS.iloc[11,6],LEARNAFRCA.iloc[11,6],
                                           LINKASSURE.iloc[11,6],INTBREW.iloc[11,6],GUINNESS.iloc[11,6],HONYFLOUR.iloc[11,6],
                                           GLAXOSMITH.iloc[11,6],FBNH.iloc[11,6],AFRIPRUD.iloc[11,6],COURTVILLE.iloc[11,6],
                                           CUSTODIAN.iloc[11,6],CWG.iloc[11,6],DANGCEM.iloc[11,6],ELLAHLAKES.iloc[11,6],
                                           ETERNA.iloc[11,6],ETI.iloc[11,6],FLOURMILL.iloc[11,6],LASACO.iloc[11,6],
                                           MANSARD.iloc[11,6],MBENEFIT.iloc[11,6],MEYER.iloc[11,6],MORISON.iloc[11,6],MRS.iloc[11,6],
                                           NEM.iloc[11,6],UACN.iloc[11,6],UCAP.iloc[11,6],UNIVINSURE.iloc[11,6],WAPIC.iloc[11,6],
                                           VITAFOAM.iloc[11,6]],
                             
                             
                             'Book_to_market_ratio':[AIICO.iloc[11,27],FTNCOCOA.iloc[11,27],LIVINGTRUST.iloc[11,27],
                                                     ABCTRANS.iloc[11,27],ACADEMY.iloc[11,27],AFRINSURE.iloc[11,27],
                                                     AFROMEDIA.iloc[11,7],AIRTELAFRI.iloc[11,27],ALEX.iloc[11,27],BERGER.iloc[11,27],
                                                     BETAGLAS.iloc[11,27],BUACEMENT.iloc[11,27],BUAFOODS.iloc[11,27],
                                                     CADBURY.iloc[11,27],CAP.iloc[11,27],CAVERTON.iloc[11,27],CHAMPION.iloc[11,27],
                                                     CHAMS.iloc[11,27],CHELLARAM.iloc[11,27],CHIPLC.iloc[11,27],CILEASING.iloc[11,27],
                                                     ETRANZACT.iloc[11,27],CONOIL.iloc[11,27],TRANSCORP.iloc[11,27],
                                                     UNILEVER.iloc[11,27],UPDCREIT.iloc[11,27],UPL.iloc[11,27],VERITASKAP.iloc[11,27],
                                                     ZENITHBANK.iloc[11,27],WAPCO.iloc[11,27],UPDC.iloc[11,27],UBN.iloc[11,27],
                                                     PZ.iloc[11,27],LIVESTOCK.iloc[11,27],JAIZBANK.iloc[11,27],FCMB.iloc[11,27],
                                                     FIDELITYBK.iloc[11,27],UBA.iloc[11,27],TOTAL.iloc[11,27],STANBIC.iloc[11,27],
                                                     SOVRENINS.iloc[11,27],RTBRISCOE.iloc[11,27],SCOA.iloc[11,27],
                                                     REDSTAREX.iloc[11,7],
                                                     REGALINS.iloc[11,7],PRESCO.iloc[11,7],PHARMDEKO.iloc[11,7],OANDO.iloc[11,7],
                                                     OKOMUOIL.iloc[11,7],NNFM.iloc[11,7],NGXGROUP.iloc[11,7],NESTLE.iloc[11,7],
                                                     NEIMETH.iloc[11,7],NB.iloc[11,7],MULTIVERSE.iloc[11,7],NAHCO.iloc[11,7],
                                                     NASCON.iloc[11,7],MTNN.iloc[11,7],MAYBAKER.iloc[11,7],MCNICHOLS.iloc[11,7],
                                                     LEARNAFRCA.iloc[11,7],LINKASSURE.iloc[11,7],INTBREW.iloc[11,7],
                                                     GUINNESS.iloc[11,7],HONYFLOUR.iloc[11,7],GLAXOSMITH.iloc[11,7],FBNH.iloc[11,7],
                                                     AFRIPRUD.iloc[11,7],COURTVILLE.iloc[11,7],CUSTODIAN.iloc[11,7],CWG.iloc[11,7],
                                                     DANGCEM.iloc[11,7],ELLAHLAKES.iloc[11,7],ETERNA.iloc[11,7],ETI.iloc[11,7],
                                                     FLOURMILL.iloc[11,7],LASACO.iloc[11,7],MANSARD.iloc[11,7],MBENEFIT.iloc[11,7],
                                                     MEYER.iloc[11,7],MORISON.iloc[11,7],MRS.iloc[11,7],NEM.iloc[11,7],
                                                     UACN.iloc[11,7],UCAP.iloc[11,7],UNIVINSURE.iloc[11,7],WAPIC.iloc[11,7],
                                                     VITAFOAM.iloc[11,7]],
                             
                            
                            'Change_in_Price':price12},
                            
                             
                             index=[['AIICO', 'FTNCOCOA', 'LIVINGTRUST', 'ABCTRANS', 'ACADEMY','AFRINSURE', 'AFROMEDIA',
                                     'AIRTELAFRI', 'ALEX', 'BERGER', 'BETAGLAS','BUACEMENT', 'BUAFOODS', 'CADBURY', 'CAP','CAPHOTEL',
                                     'CAVERTON', 'CHAMPION','CHAMS', 'CHELLARAM', 'CHIPLC', 'CILEASING', 'ETRANZACT', 'CONOIL',
                                     'TRANSCORP', 'UNILEVER', 'UPDCREIT', 'UPL', 'VERITASKAP', 'ZENITHBANK','WAPCO', 'UPDC', 
                                     'UBN', 'PZ', 'LIVESTOCK','JAIZBANK', 'FCMB','FIDELITYBK', 'UBA', 'TOTAL', 'STANBIC', 
                                     'SOVRENINS', 'RTBRISCOE','SCOA''REDSTAREX', 'REGALINS', 'PRESCO', 'PHARMDEKO', 'OANDO',
                                     'OKOMUOIL', 'NNFM', 'NGXGROUP', 'NESTLE', 'NEIMETH', 'NB', 'MULTIVERSE','NAHCO', 'NASCON',
                                     'MTNN', 'MAYBAKER', 'MCNICHOLS', 'LEARNAFRCA','LINKASSURE', 'INTBREW', 'GUINNESS',
                                     'HONYFLOUR', 'GLAXOSMITH','FBNH','AFRIPRUD', 'COURTVILLE', 'CUSTODIAN', 'CWG', 'DANGCEM',
                                     'ELLAHLAKES','ETERNA', 'ETI', 'FLOURMILL', 'LASACO', 'MANSARD', 'MBENEFIT', 'MEYER',
                                     'MORISON', 'MRS', 'NEM', 'UACN', 'UCAP', 'UNIVINSURE', 'WAPIC','VITAFOAM']]
                            )

In [86]:
august_2022

,Market_Cap,Book_to_market_ratio,Change_in_Price
AIICO,2.013290e+10,2.129921,-0.035088
FTNCOCOA,6.600000e+08,-3.227576,0.000000
LIVINGTRUST,7.150000e+09,0.408973,0.191667
ABCTRANS,5.138870e+08,1.159302,0.033333
ACADEMY,1.270080e+09,0.312264,0.135135
...,...,...,...
UACN,3.169430e+10,1.624204,-0.017857
UCAP,7.200000e+10,0.331239,0.081081
UNIVINSURE,3.200000e+09,3.068106,0.000000
WAPIC,9.116884e+09,2.269964,-0.073171


In [87]:
SL11=august_2022[(august_2022['Market_Cap']<=2000000000)&(august_2022['Book_to_market_ratio']>1)]

SM11=august_2022[(august_2022['Market_Cap']>2000000000) & (august_2022['Market_Cap']<11000000000) & (august_2022['Book_to_market_ratio']>1)]

SH11=august_2022[(august_2022['Market_Cap']>=11000000000)&(august_2022['Book_to_market_ratio']>1)]

BL11=august_2022[(august_2022['Market_Cap']<=2000000000) & (august_2022['Book_to_market_ratio']<1)]

BH11=august_2022[(august_2022['Market_Cap']>=11000000000)&(august_2022['Book_to_market_ratio']<1)]

BM11=august_2022[(august_2022['Market_Cap']>2000000000) & (august_2022['Market_Cap']<11000000000) & (august_2022['Book_to_market_ratio']<1)]


In [88]:
SLPRICE11=SL11['Change_in_Price'].mean()

SMPRICE11=SM11['Change_in_Price'].mean()

SHPRICE11=SH11['Change_in_Price'].mean()

BLPRICE11=BL11['Change_in_Price'].mean()

BMPRICE11=BM11['Change_in_Price'].mean()

BHPRICE11=BH11['Change_in_Price'].mean()



In [89]:
HML11=(1/3*(SLPRICE11+SMPRICE11+SHPRICE11)) - (1/3*(BLPRICE11 + BMPRICE11 + BHPRICE11))

SMB11=(1/2*(SHPRICE11 + BHPRICE11)) - (1/2*(SLPRICE11 +BLPRICE11))

HML11,SMB11

(-0.009649689357421401, 0.021412463130563365)

In [90]:
fff=pd.DataFrame({'HML':[HML,HML1,HML2,HML3,HML4,HML5,HML6,HML7,HML8,HML9,HML10,HML11,],
                 'SMB':[SMB,SMB1,SMB2,SMB3,SMB4,SMB5,SMB6,SMB7,SMB8,SMB9,SMB10,SMB11,],
                  'datetime':['September 30, 2021','October 31, 2021','November 30, 2021','December 31, 2021','January 31, 2022',
                       'February 28, 2022','March 31, 2022','April 30, 2022','May 31, 2022','June 30, 2022','July 30, 2022',
                       'August 31, 2022']})

In [91]:
fff['datetime']=pd.to_datetime(fff['datetime'])

In [92]:
fff

,HML,SMB,datetime
0,-0.000229,0.009393,2021-09-30
1,0.039417,0.088441,2021-10-31
2,-0.011932,-0.017114,2021-11-30
3,0.018859,-0.075099,2021-12-31
4,-0.054132,-0.034212,2022-01-31
5,-0.144912,-0.374175,2022-02-28
6,-0.007865,0.061634,2022-03-31
7,-0.153147,0.091260,2022-04-30
8,0.061007,-0.016428,2022-05-31
9,0.005401,-0.020103,2022-06-30


In [93]:
ff=fff.merge(risk_free,on='datetime')
ff.columns=ff.columns.str.replace('TreasuryBill','Rf')

In [94]:
ff=ff.set_index('datetime')

In [95]:
ff

,HML,SMB,Rf
datetime,,,
2021-09-30,-0.000229,0.009393,0.000000
2021-10-31,0.039417,0.088441,0.000000
2021-11-30,-0.011932,-0.017114,0.000000
2021-12-31,0.018859,-0.075099,-0.004000
2022-01-31,-0.054132,-0.034212,0.000000
2022-02-28,-0.144912,-0.374175,0.000000
2022-03-31,-0.007865,0.061634,-0.297189
2022-04-30,-0.153147,0.091260,-0.005714
2022-05-31,0.061007,-0.016428,0.419540


### Market Data

In [96]:
market=pd.read_csv("/Users/USER/Documents/DATASETS/NSE_ASI_Index_MONTHLY_market_prices_2013_2021.csv")[['datetime','asi']]

In [97]:
market['datetime']=pd.to_datetime(market['datetime'])

In [98]:
market=market.set_index('datetime')

In [99]:
market=market['asi'].pct_change().dropna(how='all')

In [100]:
market=market.resample('1M').first()

In [101]:
market

datetime
2013-10-31    0.028363
2013-11-30    0.034503
2013-12-31    0.061878
2014-01-31   -0.018330
2014-02-28   -0.024962
                ...   
2022-05-31    0.080546
2022-06-30   -0.033923
2022-07-31   -0.027931
2022-08-31   -0.010596
2022-09-30   -0.013546
Freq: M, Name: asi, Length: 108, dtype: float64

In [102]:
market=market.reset_index()
market

,datetime,asi
0,2013-10-31,0.028363
1,2013-11-30,0.034503
2,2013-12-31,0.061878
3,2014-01-31,-0.018330
4,2014-02-28,-0.024962
...,...,...
103,2022-05-31,0.080546
104,2022-06-30,-0.033923
105,2022-07-31,-0.027931
106,2022-08-31,-0.010596


In [103]:
gff=ff.merge(market,on='datetime')

In [104]:
gff['Mkt-Rf']=gff['asi']-gff['Rf']
gff

,datetime,HML,SMB,Rf,asi,Mkt-Rf
0,2021-09-30,-0.000229,0.009393,0.000000,0.025537,0.025537
1,2021-10-31,0.039417,0.088441,0.000000,0.045186,0.045186
2,2021-11-30,-0.011932,-0.017114,0.000000,0.028770,0.028770
3,2021-12-31,0.018859,-0.075099,-0.004000,-0.012292,-0.008292
4,2022-01-31,-0.054132,-0.034212,0.000000,0.091492,0.091492
5,2022-02-28,-0.144912,-0.374175,0.000000,0.016512,0.016512
6,2022-03-31,-0.007865,0.061634,-0.297189,-0.009053,0.288136
7,2022-04-30,-0.153147,0.091260,-0.005714,0.056924,0.062638
8,2022-05-31,0.061007,-0.016428,0.419540,0.080546,-0.338995
9,2022-06-30,0.005401,-0.020103,-0.024291,-0.033923,-0.009632


In [106]:
gff.to_csv("/Users/USER/Documents/DATASETS/Fama_french_factors.csv",index=False)
